## Блок импорта всех необходимых библиотек

Этот блок импортирует основные библиотеки, используемые для обработки, анализа и очистки данных.  

**Действия и назначение библиотек:**
- `os` — работа с файловой системой 
- `pandas` — работа с табличными данными (чтение, объединение, фильтрация, сохранение)  
- `numpy` — числовые операции, работа с массивами и значениями `NaN`  
- `requests` — выполнение HTTP-запросов к внешним API (например, Yandex Geocoder)  
- `webbrowser` — стандартный модуль Python, позволяющий автоматически открывать HTML-файлы (например, сохранённые интерактивные карты) в браузере по умолчанию
- `folium` — построение интерактивных карт (в том числе с базой Яндекс-тайлов)  
- `MarkerCluster` из `folium.plugins` — объединение множества точек в кластеры для наглядного отображения на карте

In [2]:
# Импорт необходимых библиотек
import os
import pandas as pd
import numpy as np
import requests
import webbrowser
import folium
from folium.plugins import MarkerCluster

## Блок вспомогательных функций

Этот блок содержит пользовательские функции, которые используются в разных этапах проекта.  

**Функция `yandex_folium_map(points, zoom_start=4, lang="ru_RU")`:**  
Создаёт интерактивную карту с использованием библиотеки `folium` и тайлов Яндекс.Карт. Принимает список координат (с возможными подписями `popup`) и визуализирует их на карте, группируя близко расположенные точки в кластеры для удобства восприятия.

**Функция `geocode(adress, api_key)`:**  
Выполняет геокодирование адреса с использованием API Яндекс.Карт. На вход принимает строку с адресом и ключ API, отправляет HTTP-запрос к сервису `geocode-maps.yandex.ru` и возвращает координаты объекта (долготу и широту), а также структурированную информацию об адресе. Используется для автоматического получения географических координат по текстовым адресам при подготовке и обогащении датасета.

**Функция `process_geocoding(df, rows, api_key, use_en=True, use_ru=True)`:**   
Выполняет геокодирование выбранных адресов в датафрейме с помощью API Яндекс.Карт.  
По каждому адресу вызывает `geocode()` и добавляет координаты (`lat`, `lon`) и нормализованные поля адреса.  
Позволяет выбрать направления обработки:  
`use_en` — геокодирование столбца `address` (результаты в `_en`),  
`use_ru` — геокодирование столбца `address_rus` (результаты в `_ru`).  
Используется для автоматического дополнения данных координатами и структурой адресов.

**Функция `split_to_clean_and_left(df, rule, cleaned_path="data/cleaned_data.csv", source_suffix="en")`:**  
Переносит строки из датасета `df` в финальный очищенный файл `cleaned_data.csv` на основе заранее заданного правила (булевой маски).  
По выбранным строкам функция извлекает геоданные из полей с суффиксом (`_en` или `_ru`), переименовывает их в универсальные колонки (`geo_address`, `geo_lon`, `geo_lat` и др.) и добавляет в итоговый набор очищенных данных. Возвращает датафрейм `df_left_to_clean`, содержащий только те записи, которые не были перенесены и требуют дальнейшей обработки или повторного геокодирования.

**Функция `norm(string)`:**  
Возвращает очищенную нормализованную строку.  
Преобразует входной текст в нижний регистр, удаляет лишние пробелы по краям, заменяет символ `ё` на `е`, а также схлопывает повторяющиеся пробелы и запятые.  
Используется для предварительной очистки текстовых данных (например, адресов) перед сравнением, нормализацией или объединением.

In [3]:
def yandex_folium_map(points, zoom_start=4, lang="ru_RU"):
    """
    Создаёт интерактивную карту с точками на основе тайлов Яндекс.Карт.

    Параметры:
    points: list - Список кортежей (lat, lon) или (lat, lon, popup).
    zoom_start: int, optional - Начальный уровень приближения. По умолчанию 4.
    lang: str, optional - Язык интерфейса карты. По умолчанию 'ru_RU'.

    Возвращает: folium.Map - Объект интерактивной карты.
    """
    # Берём средние координаты всех точек для вычисления центра карты
    lat_mean = sum(p[0] for p in points) / len(points)
    lon_mean = sum(p[1] for p in points) / len(points)
    center = (lat_mean, lon_mean)

    # Создаём карту folium без встроенных тайлов
    atm_map = folium.Map(location=center, zoom_start=zoom_start, tiles=None, crs="EPSG3395")

    # Добавляем слой обычной карты Яндекса (улицы, дороги, города)
    yandex_map = folium.TileLayer(
        tiles=f"https://core-renderer-tiles.maps.yandex.net/tiles?l=map&x={{x}}&y={{y}}&z={{z}}&scale=1&lang={lang}",
        attr="© Yandex",
        name="Yandex Map",
        overlay=False,
        control=True
    ).add_to(atm_map)

    # Создаём кластер маркеров — точки группируются при приближении/отдалении
    cluster = MarkerCluster().add_to(atm_map)
    
    # Добавляем маркеры на карту
    for point in points:
        if len(point) >= 3: 
            # Если в точке есть popup, добавляем его
            folium.Marker(location=(point[0], point[1]), popup=str(point[2])).add_to(cluster)
        else:
             # Если popup нет, просто ставим метку
            folium.Marker(location=(point[0], point[1])).add_to(cluster)

    # Возвращаем готовый объект карты
    return atm_map

In [4]:
def geocode(adress, api_key):
    """
    Выполняет геокодирование адреса через API Яндекс.Карт.

    Функция отправляет HTTP-запрос к сервису геокодирования Яндекса 
    и получает координаты (широту и долготу) указанного адреса, 
    а также текстовое описание и структурированные компоненты адреса.

    Параметры:
    adress: str - Текстовый адрес (например, 'Москва, Тверская улица, 7').
    api_key: str - Ключ доступа к API Яндекс.Карт.

    Возвращает:
    dict или None
        Словарь с результатами геокодирования:
        {
            "text": str  — полный распознанный адрес,
            "lon": float — долгота,
            "lat": float — широта,
            "adress_components": list — структура адресных элементов (город, улица, дом и т.д.)
        }
        Возвращает None, если запрос неуспешен или объект не найден.
    """
    # Базовый URL для запроса к API Яндекс Геокодера
    url = "https://geocode-maps.yandex.ru/v1/"

    # Параметры запроса: ключ, адрес, формат, язык и количество результатов
    params = {
        "apikey": api_key,
        "geocode": f"{adress}, Russia", # добавляем "Russia" для повышения точности поиска
        "format": "json",               # получаем ответ в формате JSON
        "lang": "ru_RU",                # язык ответа
        "results": 1                    # запрашиваем только первый (наиболее релевантный) результат
    }

    try:
        # Отправляем GET-запрос к API
        r = requests.get(url, params=params, timeout=10)
        
        # Проверяем успешность ответа (код 200)
        r.raise_for_status()
        
        # Преобразуем JSON-ответ в словарь Python
        data = r.json()        

        # Извлекаем первый найденный объект из коллекции
        objects = data["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]

        # Если объект не найден, возвращаем None
        if not objects:
            return None
        
        # Полный адрес в человекочитаемом виде
        text = objects["metaDataProperty"]["GeocoderMetaData"]["text"]

        # Структурированные компоненты адреса (страна, регион, город, улица и т.д.)
        components = objects["metaDataProperty"]["GeocoderMetaData"]["Address"]["Components"]
        
        # Извлекаем координаты из строки "lon lat"
        pos = objects["Point"]["pos"].split()
        lon, lat = float(pos[0]), float(pos[1])

        return {"text": text, "lon": lon, "lat": lat, "adress_components": components}
    # Обработка любых ошибок (сетевых, парсинга, отсутствия ключей)
    except Exception:
        return None

In [5]:
def process_geocoding(df, rows, api_key=None, use_en= True, use_ru= True):
    """
    Геокодирует выбранные строки DataFrame по Яндекс HTTP Геокодеру.

    Параметры
    df: pandas.DataFrame - Таблица с адресами и результатами геокодирования.
    rows: iterable - Итератор или список строк, по которым выполняется геокодирование.
    api_key: str | None - Ключ API Яндекс Геокодера. Если None — функция ничего не делает и вернёт df без изменений.
    use_en: bool - Геокодировать ли столбец 'address', сохранять в *_en.
    use_ru: bool - Геокодировать ли столбец 'address_rus', сохранять в *_ru.

    Возвращает
    ----------
    pandas.DataFrame
        Исходный df с добавленными колонками:СИльно
        - при use_en:  geo_address_en, geo_lon_en, geo_lat_en, <component>_en
        - при use_ru:  geo_address_ru, geo_lon_ru, geo_lat_ru, <component>_ru
    """
    # Проверяем, что ключ задан
    if api_key is not None:
        for idx, row in rows:
            # EN геокодирование
            if use_en:
                # Получение адреса на английском языке
                address_en = row.get("address")

                # Результат после геокодирования по address 
                result_en = geocode(address_en, api_key)
                if not result_en:
                    print(f"Геокодер не вернул результат для индекса {idx}!")
                else:
                    # Сохраняем результаты в датафрейм
                    df.at[idx, "geo_address_en"] = result_en["text"]
                    df.at[idx, "geo_lon_en"] = result_en["lon"]
                    df.at[idx, "geo_lat_en"] = result_en["lat"]
                    for kind_dict in result_en["adress_components"]:
                        component = kind_dict["kind"]
                        value = kind_dict["name"]
                        df.at[idx, f"{component}_en"] = value
            
            # RU геокодирование
            if use_ru:
                # Получение адреса на русском языке
                address_ru = row.get("address_rus")

                # Проверка наличия русского адреса
                if pd.notna(address_ru):
                    # Результат после геокодирования по address_rus
                    result_ru = geocode(address_ru, api_key)
                    if not result_ru:
                        print(f"Геокодер не вернул результат для индекса {idx}!")
                    else:
                        # Сохраняем результаты в датафрейм
                        df.at[idx, "geo_address_ru"] = result_ru["text"]
                        df.at[idx, "geo_lon_ru"] = result_ru["lon"]
                        df.at[idx, "geo_lat_ru"] = result_ru["lat"]
                        for kind_dict in result_ru["adress_components"]:
                            component = kind_dict["kind"]
                            value = kind_dict["name"]
                            df.at[idx, f"{component}_ru"] = value
    return df

In [6]:
def split_to_clean_and_left(df, rule, cleaned_path="data/cleaned_data.csv", source_suffix="en") -> pd.DataFrame:
    """
    Переносит строки из общего датасета 'df' в файл 'cleaned_data.csv' на основе заданного правила (булевой маски).
    Используется для формирования финального очищенного набора данных.

    Параметры
    df: pd.DataFrame - Исходный датафрейм, содержащий все записи (train + test) с результатами геокодирования.
    rule: pd.Series[bool] - Булева маска (True — строки, которые нужно перенести в clean, False — оставить для дальнейшей обработки).
    cleaned_path: str, optional (по умолчанию "data/cleaned_data.csv") - Путь к CSV-файлу, куда добавляются обработанные строки.
    source_suffix: str, optional (по умолчанию "en") - Суффикс источника геоданных — '"en"' или '"ru"', определяет из каких колонок брать значения ('geo_address_en' / 'geo_address_ru', и т.п.).

    Возвращает
    pd.DataFrame - Остаток исходного датафрейма ('df_left_to_clean') — строки, не попавшие в 'cleaned_data.csv'.
    """
    # Маппинг *_en/_ru → без суффиксов
    # Определяем, какие поля с суффиксом ('_en' или '_ru') нужно переименовать в базовые
    sfx = f"_{source_suffix}"
    rename_map = {f"geo_address{sfx}": "geo_address",
                  f"geo_lon{sfx}": "geo_lon",
                  f"geo_lat{sfx}": "geo_lat",
                  f"country{sfx}": "country",
                  f"province{sfx}": "province",
                  f"area{sfx}": "area",
                  f"locality{sfx}": "locality",
                  f"street{sfx}": "street",
                  f"house{sfx}": "house"}
    
    # Колонки, которые выбираем из df в зависимости от используемого суффикса
    subset_cols = ["id", "atm_group", "address", f"geo_address{sfx}", f"geo_lon{sfx}", f"geo_lat{sfx}",
                   f"country{sfx}", f"province{sfx}", f"area{sfx}", f"locality{sfx}", f"street{sfx}", f"house{sfx}", "target", "isTrain"]

    # Формируем временный датафрейм из строк, соответствующих правилу (rule) и переименовываем колонки из *_en/_ru → базовые
    df_clean_part = (df.loc[rule, subset_cols].copy().rename(columns=rename_map))


    # Подгружаем или создаём csv файл
    if os.path.exists(cleaned_path):
        df_clean = pd.read_csv(cleaned_path)
    
        # Добавляем новые очищенные строки в основной датафрейм
        df_clean = pd.concat([df_clean, df_clean_part], ignore_index=True)
    else:
        # Если файла нет — создаём его из текущих данных
        df_clean = df_clean_part.copy()

    # Сохраняем обновлённый cleaned_data.csv
    df_clean.to_csv(cleaned_path, index=False, encoding="utf-8")
    
    # Формируем датафрейм с оставшимися строками, которые не вошли в clean
    df_left_to_clean = df.loc[~rule].reset_index(drop=True)
    
    # Возвращаем остаток
    return df_left_to_clean

In [7]:
def norm(string) -> str:
    """
    Возвращает очищенную строку: приводит к нижнему регистру, удаляет лишние пробелы, заменяет ё→е и схлопывает пробелы и запятые.
    """
    # Обрабатываем NaN
    if pd.isna(string):
        return ""
    
    # Возвращаем обработанную строку
    return " ".join(str(string).strip().lower().replace("ё", "е").replace(",", " ").split())

## Блок чтения исходных данных и их первичный анализ

Данный блок выполняет первичную загрузку и обзор данных, необходимых для анализа популярности банкоматов. Загрузка происходит напрямую из GitHub через ссылки на RAW-версии файлов.  

### Этапы:
1. Загрузка данных  
   Используются файлы `train.csv` и `test.csv`, содержащие исходные данные по банкоматам. Данные читаются в формате `pandas.DataFrame`
2. Добавление признака принадлежности  
   Добавляется булев столбец `isTrain`, позволяющий различать строки обучающей (`True`) и тестовой (`False`) выборок
3. Объединение данных  
   Оба датафрейма объединяются в единый `all_data` для последующей унифицированной обработки и визуализации
4. Первичный анализ  
   Выполняется вывод общей информации о датасете (`info()`), а также проверка наличия пропусков в признаках (`isnull().sum()`)
5. Базовая визуализация  
   Для наглядного понимания географического распределения объектов создаётся интерактивная карта с помощью библиотеки `folium` и тайлов Яндекс.Карт.  
   Каждая точка на карте соответствует банкомату, а при клике отображается всплывающее окно с ID и адресом объекта

In [125]:
# Загрузка исходных данных из Github

# Ссылки на raw-версии файлов из репозитория GitHub
train_url = "https://raw.githubusercontent.com/Khamoon7/GeoATM-popularity/refs/heads/main/data/train.csv"
test_url  = "https://raw.githubusercontent.com/Khamoon7/GeoATM-popularity/refs/heads/main/data/test.csv"

# Чтение файлов 
train_df = pd.read_csv(train_url, index_col=0)
test_df = pd.read_csv(test_url, index_col=0)

# Добавление колонки isTrain для удобства:
# True  → строка относится к train (обучающая выборка)
# False → строка относится к test (тестовая выборка)
train_df['isTrain'] = True
test_df['isTrain'] = False

# Объединение train и test в один DataFrame
all_data = pd.concat([train_df, test_df], ignore_index=True)

# Отображение объединённых данных
display(all_data)

,id,atm_group,address,address_rus,lat,long,target,isTrain
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500,True
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710,True
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540,True
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035,True
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277,True
...,...,...,...,...,...,...,...,...
8760,7805.0,5478.0,"D. 12, PROEZD ZAREVYI MOSKVA G","Заревый проезд, 12, Москва, Россия, 127282",55.886488,37.656829,NaN,False
8761,7395.0,5478.0,"D. 2B, UL. MIRA SOCHI G","улица Мира, 4, Адлерский район, Сочи, Краснода...",43.438310,39.912211,NaN,False
8762,3339.0,8083.0,"KARMANITSKIY, 9 Moskva","Карманицкий переулок, 9, Москва, Россия, 119002",55.748275,37.583571,NaN,False
8763,675.0,1942.0,3A SHARAPOVSKIY WAY MYTISHCHI,"Шараповская улица, 1, Мытищи, Московская облас...",55.910992,37.748466,NaN,False


In [20]:
# Отображение общей информации о датасете
all_data.info()

# Просмотр пропусков в датасете
display(all_data.isnull().sum()[lambda x: x > 0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8765 entries, 0 to 8764
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           8765 non-null   float64
 1   atm_group    8765 non-null   float64
 2   address      8765 non-null   object 
 3   address_rus  8345 non-null   object 
 4   lat          8345 non-null   float64
 5   long         8345 non-null   float64
 6   target       6261 non-null   float64
 7   isTrain      8765 non-null   bool   
dtypes: bool(1), float64(5), object(2)
memory usage: 488.0+ KB


address_rus     420
lat             420
long            420
target         2504
dtype: int64

In [128]:
# Очищаем датасет от выбросов в координатах
df_points = all_data.dropna(subset=["lat", "long"]).copy()

# Создаём столбец с текстом для всплывающей подсказки
df_points["__popup__"] = df_points.apply(lambda row: f"ID: {row['id']}. {row['address_rus']}", axis=1)

# Формируем список точек (lat, lon, popup)
points = list(zip(df_points["lat"], df_points["long"], df_points["__popup__"]))

# Создаем карту и отображаем ее
atm_map = yandex_folium_map(points)
atm_map.save("data/yandex_map_raw_dataset.html")
webbrowser.open(f"file://{os.path.abspath('data/yandex_map_raw_dataset.html')}")

True

## Подсказки и комментарии от организаторов

Организаторы конкурса предоставили ряд пояснений и рекомендаций по работе с исходными данными и особенностями формирования выборок.  
Эти замечания помогают корректно интерпретировать структуру датасета и учитывать его ограничения при анализе и построении моделей.

### Общие сведения о данных
1. Все банкоматы относятся к одному временному интервалу наблюдений и сохраняли постоянное местоположение в течение всего периода сбора данных, необходимого для расчёта таргета.
2. В системах банка изначально хранилось только поле `address`.  
Поля `address_rus`, `lat`, `long` были восстановлены на основе `address`.  
Для части объектов определить адрес и координаты с точностью до дома не удалось.  
Организаторы рекомендуют попробовать исследовать механизм восстановления с помощью поисковиков или инструментов вроде `selenium`.
3. Поле `id` формировалось из комбинации `atm_group` (группа банкоматов) и уникального идентификатора внутри группы.  
При сортировке `X.sort_values('id')` можно увидеть логику построения.  
Наибольшие значения `target` соответствуют банкоматам Росбанка, остальные — банкам-партнёрам.

### Описание целевого признака (`target`)
4. `target` отражает популярность банкомата — функцию от количества операций за единицу времени.  
5. Таргет обрезан сверху — удалены аномально популярные устройства, чья активность не объясняется геолокацией.

### Особенности структуры выборок
6. Для визуализации рекомендуется использовать библиотеку `folium`, позволяющую строить интерактивные карты расположения банкоматов.
7. Точность восстановления адресов можно повысить с помощью эвристик и парсинга.
8. Для банкоматов банков-партнёров адреса доступны только на латинице, так как они были извлечены из транзакционных данных.  
Можно либо восстановить адреса на кириллице, либо использовать эти строки для генерации геопризнаков (например, через парсинг или обработку текста).

## Структура объединённого датафрейма
Датасет содержит информацию о банкоматах Росбанка и его партнеров. Каждая запись соответствует одному банкомату и включает его идентификатор, группу, адрес, географические координаты и целевое значение (`target`), отражающее индекс популярности устройства.

Исходные данные представлены в виде двух файлов — `train.csv` и `test.csv`, объединённых в единый датафрейм `all_data`.  
Файлы предоставлены организаторами соревнования и имеют идентичную структуру  
  
**Размер:** 8 765 строк × 8 столбцов  
Данные: 
| Колонка        | Тип данных | Кол-во непустых | Пропусков | Описание |
|----------------|-------------|------------------|------------|-----------|
| `id`           | float64     | 8 765 | 0 | Уникальный идентификатор банкомата |
| `atm_group`    | float64     | 8 765 | 0 | Код группы банкоматов (банк-владелец) |
| `address`      | object      | 8 765 | 0 | Исходный адрес (из банковской системы) |
| `address_rus`  | object      | 8 345 | **420** | Восстановленный адрес на русском языке |
| `lat`          | float64     | 8 345 | **420** | Географическая широта банкомата |
| `long`         | float64     | 8 345 | **420** | Географическая долгота банкомата |
| `target`       | float64     | 6 261 | **2 504** | Индекс популярности |
| `isTrain`      | bool        | 8 765 | 0 | Метка принадлежности к обучающей или тестовой выборке |

### Выводы предварительного анализа
1. Пропуски по ключевым геополям.  
   Подтверждено наличие пропусков в `address_rus`, `lat`, `long` — всего по **420 записей** в каждом столбце. Это совпадает с комментариями организаторов о неполном восстановлении координат и адресов.

2. Несовпадения координат и адресов.  
   По визуализации на карте видно, что часть точек расположена не точно по адресу, что отражает эвристический характер восстановления координат.

3. Неполные адреса на русском.  
   В некоторых строках `address_rus` отсутствуют детали (например, номер дома), что может влиять на точность геокодирования и группировки по адресу.


## Блок геокодирования и очистки данных
В данном блоке реализована полная цепочка автоматической и полуавтоматической очистки адресных данных: от массового геокодирования до финального формирования очищенного датасета.  
Ключевые функции обеспечивают последовательные этапы — геокодирование, проверку совпадений, склейку неполных адресов и обработку ручных правок.
#### 1) Геокодирование исходных адресов
- Выполняется массовое геокодирование всех строк через Yandex HTTP Геокодер (`process_geocoding`).
- Адреса обрабатываются в двух языковых вариантах — **русском и английском** (`use_ru=True`, `use_en=True`).
- В результат добавляются поля: `geo_address_ru/en`, координаты `geo_lat/geo_lon`, и компоненты (`country`, `province`, `area`, `locality`, `street`, `house`).

#### 2) Сужение структуры и сохранение промежуточного результата
- После геокодирования удаляются вспомогательные поля (`metro`, `route`, `other` и т.п.), оставляются только ключевые компоненты.
- Данные сохраняются в CSV для последующей валидации и очистки.

#### 3) Базовая автоматическая верификация
- Проверяется совпадение адресов на русском и английском (`geo_address_en == geo_address_ru`).
- Совпадающие строки считаются корректными и добавляются в очищенные данные.
- Остальные строки выделяются в отдельный датафрейм для дообработки.

#### 4) Дополнительное правило склейки адресов (`apply_rule`)
- Выполняется **нормализация** текстовых полей (строчные буквы, замена `ё→е`, удаление лишних пробелов).
- Проверяются совпадения верхних компонентов (`country`, `province`, `area`, `locality`).
- Если совпадают — объединяются неполные части (`street`, `house`) из RU и EN версий.
- Собирается итоговый адрес `geo_address`, который добавляется в очищенные данные.

#### 5) Ручные правки и доочистка
- Для сложных кейсов используется словарь `updates = {id: new_address}`.
- Исправленные строки геокодируются заново и сохраняются обратно.
- Строки, которые невозможно корректно восстановить, удаляются из набора.

#### 6) Финализация и сохранение результата
- Все очищенные и восстановленные данные объединяются в общий датасет `cleaned_data.csv`.
- Финальная версия сохраняется в `geo_data_cleaned.csv` и используется для последующего анализа или визуализации.


In [ ]:
# API ключ для Yandex HTTP Геокодер
# Можно получить тут - https://developer.tech.yandex.ru/services/3
API_KEY = None

# Путь для сохранения датасета
OUTPUT_PATH = "data/all_data_geocoding_results.csv"

# Геокодирование датафрейма
all_data = process_geocoding(all_data, all_data.iterrows(), API_KEY, use_en=True, use_ru=True)

# Отображение датафрейма
display(all_data)

# В результате геокодирования появилось множество вспомогательных полей (other, route, metro и др.), большинство из которых пустые или неинформативны.
# Удалим их, а также исходные координаты (lat, long) (из-за их неточности), перед сохранением данных в csv файл
all_data = all_data[["id", "atm_group", "address", "address_rus", "geo_address_en", "geo_address_ru", 
                     "geo_lon_en", "geo_lon_ru", "geo_lat_en", "geo_lat_ru", "country_en", "country_ru", 
                      "province_en", "province_ru", "area_en", "area_ru", "locality_en", "locality_ru", 
                      "street_en", "street_ru", "house_en", "house_ru", "target", "isTrain"]]

# Сохранение в csv файл
all_data.to_csv(OUTPUT_PATH, index=False, encoding="utf-8")

,id,atm_group,address,address_rus,geo_address_en,geo_address_ru,geo_lon_en,geo_lon_ru,geo_lat_en,geo_lat_ru,...,area_en,area_ru,locality_en,locality_ru,street_en,street_ru,house_en,house_ru,target,isTrain
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...","Россия, Сахалинская область, Южно-Сахалинск, у...","Россия, Сахалинская область, Южно-Сахалинск, у...",142.738319,142.738319,46.940995,46.940995,...,городской округ Южно-Сахалинск,городской округ Южно-Сахалинск,Южно-Сахалинск,Южно-Сахалинск,улица А.О. Емельянова,улица А.О. Емельянова,34,34,0.011500,True
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...","Россия, Сахалинская область, Южно-Сахалинск, К...","Россия, Сахалинская область, Южно-Сахалинск, К...",142.753348,142.753348,46.937347,46.937347,...,городской округ Южно-Сахалинск,городской округ Южно-Сахалинск,Южно-Сахалинск,Южно-Сахалинск,Комсомольская улица,Комсомольская улица,259,259,0.029710,True
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...","Россия, Сахалинская область, Южно-Сахалинск, 32","Россия, Сахалинская область, Южно-Сахалинск, К...",142.722751,142.741149,47.060423,46.959413,...,городской округ Южно-Сахалинск,городской округ Южно-Сахалинск,Южно-Сахалинск,Южно-Сахалинск,NaN,Коммунистический проспект,32,NaN,0.009540,True
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...","Россия, Москва, Ленинградский проспект, 76А","Россия, Москва, Ленинградский проспект, 76А",37.515164,37.515164,55.805837,55.805837,...,NaN,NaN,Москва,Москва,Ленинградский проспект,Ленинградский проспект,76А,76А,-0.094035,True
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...","Россия, Красноярский край, Норильск, Гвардейск...","Россия, Красноярский край, Норильск, Гвардейск...",88.211228,88.211228,69.343544,69.343544,...,городской округ Норильск,городской округ Норильск,Норильск,Норильск,Гвардейская площадь,Гвардейская площадь,2,2,0.079277,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8760,7805.0,5478.0,"D. 12, PROEZD ZAREVYI MOSKVA G","Заревый проезд, 12, Москва, Россия, 127282","Россия, Москва, Заревый проезд, 12","Россия, Москва, Заревый проезд, 12",37.656820,37.656820,55.886524,55.886524,...,NaN,NaN,Москва,Москва,Заревый проезд,Заревый проезд,12,12,NaN,False
8761,7395.0,5478.0,"D. 2B, UL. MIRA SOCHI G","улица Мира, 4, Адлерский район, Сочи, Краснода...",Россия,"Россия, Краснодарский край, городской округ Со...",99.505405,40.000659,61.698657,43.406042,...,NaN,городской округ Сочи,NaN,село Весёлое,NaN,улица Мира,NaN,2/2,NaN,False
8762,3339.0,8083.0,"KARMANITSKIY, 9 Moskva","Карманицкий переулок, 9, Москва, Россия, 119002","Россия, Москва, Карманицкий переулок, 9","Россия, Москва, Карманицкий переулок, 9",37.583257,37.583257,55.748265,55.748265,...,NaN,NaN,Москва,Москва,Карманицкий переулок,Карманицкий переулок,9,9,NaN,False
8763,675.0,1942.0,3A SHARAPOVSKIY WAY MYTISHCHI,"Шараповская улица, 1, Мытищи, Московская облас...","Россия, Московская область, Мытищи, Шараповски...","Россия, Московская область, Мытищи, Шараповска...",37.763081,37.748151,55.917364,55.911093,...,городской округ Мытищи,городской округ Мытищи,Мытищи,Мытищи,Шараповский проезд,Шараповская улица,3А,1,NaN,False


In [20]:
# Путь для чтения датасета после геокодирования
INPUT_PATH = "data/all_data_geocoding_results.csv"

# Чтение файла
all_data = pd.read_csv(INPUT_PATH, encoding="utf-8")

# Отображение датасета
display(all_data)

,id,atm_group,address,address_rus,geo_address_en,geo_address_ru,geo_lon_en,geo_lon_ru,geo_lat_en,geo_lat_ru,...,area_en,area_ru,locality_en,locality_ru,street_en,street_ru,house_en,house_ru,target,isTrain
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...","Россия, Сахалинская область, Южно-Сахалинск, у...","Россия, Сахалинская область, Южно-Сахалинск, у...",142.738319,142.738319,46.940995,46.940995,...,городской округ Южно-Сахалинск,городской округ Южно-Сахалинск,Южно-Сахалинск,Южно-Сахалинск,улица А.О. Емельянова,улица А.О. Емельянова,34,34,0.011500,True
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...","Россия, Сахалинская область, Южно-Сахалинск, К...","Россия, Сахалинская область, Южно-Сахалинск, К...",142.753348,142.753348,46.937347,46.937347,...,городской округ Южно-Сахалинск,городской округ Южно-Сахалинск,Южно-Сахалинск,Южно-Сахалинск,Комсомольская улица,Комсомольская улица,259,259,0.029710,True
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...","Россия, Сахалинская область, Южно-Сахалинск, 32","Россия, Сахалинская область, Южно-Сахалинск, К...",142.722751,142.741149,47.060423,46.959413,...,городской округ Южно-Сахалинск,городской округ Южно-Сахалинск,Южно-Сахалинск,Южно-Сахалинск,NaN,Коммунистический проспект,32,NaN,0.009540,True
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...","Россия, Москва, Ленинградский проспект, 76А","Россия, Москва, Ленинградский проспект, 76А",37.515164,37.515164,55.805837,55.805837,...,NaN,NaN,Москва,Москва,Ленинградский проспект,Ленинградский проспект,76А,76А,-0.094035,True
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...","Россия, Красноярский край, Норильск, Гвардейск...","Россия, Красноярский край, Норильск, Гвардейск...",88.211228,88.211228,69.343544,69.343544,...,городской округ Норильск,городской округ Норильск,Норильск,Норильск,Гвардейская площадь,Гвардейская площадь,2,2,0.079277,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8760,7805.0,5478.0,"D. 12, PROEZD ZAREVYI MOSKVA G","Заревый проезд, 12, Москва, Россия, 127282","Россия, Москва, Заревый проезд, 12","Россия, Москва, Заревый проезд, 12",37.656820,37.656820,55.886524,55.886524,...,NaN,NaN,Москва,Москва,Заревый проезд,Заревый проезд,12,12,NaN,False
8761,7395.0,5478.0,"D. 2B, UL. MIRA SOCHI G","улица Мира, 4, Адлерский район, Сочи, Краснода...",Россия,"Россия, Краснодарский край, городской округ Со...",99.505405,40.000659,61.698657,43.406042,...,NaN,городской округ Сочи,NaN,село Весёлое,NaN,улица Мира,NaN,2/2,NaN,False
8762,3339.0,8083.0,"KARMANITSKIY, 9 Moskva","Карманицкий переулок, 9, Москва, Россия, 119002","Россия, Москва, Карманицкий переулок, 9","Россия, Москва, Карманицкий переулок, 9",37.583257,37.583257,55.748265,55.748265,...,NaN,NaN,Москва,Москва,Карманицкий переулок,Карманицкий переулок,9,9,NaN,False
8763,675.0,1942.0,3A SHARAPOVSKIY WAY MYTISHCHI,"Шараповская улица, 1, Мытищи, Московская облас...","Россия, Московская область, Мытищи, Шараповски...","Россия, Московская область, Мытищи, Шараповска...",37.763081,37.748151,55.917364,55.911093,...,городской округ Мытищи,городской округ Мытищи,Мытищи,Мытищи,Шараповский проезд,Шараповская улица,3А,1,NaN,False


In [22]:
# Правило: совпадают ли адреса на английском и русском
rule_equal = all_data["geo_address_en"] == all_data["geo_address_ru"]

# Разделяем данные на очищенные и оставшиеся по условию rule_equal
df_left_to_clean = split_to_clean_and_left(all_data, rule_equal, source_suffix="en")

# Выводим оставшиеся строки для дальнейшей обработки
display(df_left_to_clean)

,id,atm_group,address,address_rus,geo_address_en,geo_address_ru,geo_lon_en,geo_lon_ru,geo_lat_en,geo_lat_ru,...,area_en,area_ru,locality_en,locality_ru,street_en,street_ru,house_en,house_ru,target,isTrain
0,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...","Россия, Сахалинская область, Южно-Сахалинск, 32","Россия, Сахалинская область, Южно-Сахалинск, К...",142.722751,142.741149,47.060423,46.959413,...,городской округ Южно-Сахалинск,городской округ Южно-Сахалинск,Южно-Сахалинск,Южно-Сахалинск,NaN,Коммунистический проспект,32,NaN,0.009540,True
1,2895.0,32.0,Zhivopisnaia 136 d.Velednikov,"деревня Веледниково, городской округ Истра, Мо...","Россия, Московская область, муниципальный окру...","Россия, Московская область, муниципальный окру...",37.126113,37.116025,55.801430,55.799770,...,муниципальный округ Истра,муниципальный округ Истра,деревня Веледниково,деревня Веледниково,Живописная улица,NaN,136к1,NaN,0.141469,True
2,1992.0,496.5,GORKOGO 20 TAMBOV,"улица Максима Горького, 20, Тамбов, Россия","Россия, Тамбов, 2-й переулок Горького","Россия, Тамбов, улица Максима Горького, 20",41.435134,41.454511,52.716452,52.718497,...,городской округ Тамбов,городской округ Тамбов,Тамбов,Тамбов,2-й переулок Горького,улица Максима Горького,NaN,20,-0.048436,True
3,4489.0,496.5,LENINA 56 AKSHA,"улица Ленина, 52/7, село Акша, Акшинский район...","Россия, Забайкальский край, Акшинский муниципа...","Россия, Забайкальский край, Акшинский муниципа...",113.286576,113.286909,50.280643,50.281236,...,Акшинский муниципальный округ,Акшинский муниципальный округ,село Акша,село Акша,улица Ленина,улица Ленина,54,52/7,-0.059237,True
4,4510.0,496.5,PER. BAZARNY 4 SPASSK-DALNY,NaN,"Россия, Томск, Базарный переулок, 4",NaN,84.941253,NaN,56.474668,NaN,...,городской округ Томск,NaN,Томск,NaN,Базарный переулок,NaN,4,NaN,0.039191,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3032,2948.0,496.5,MIRA 231 YUZHNO-SAKHA,"проспект Мира, 231, Южно-Сахалинск, Сахалинска...","Россия, посёлок городского типа Новоайдар, ква...","Россия, Сахалинская область, Южно-Сахалинск, п...",38.992588,142.744517,48.964270,46.943684,...,муниципальный округ Новоайдарский,городской округ Южно-Сахалинск,посёлок городского типа Новоайдар,Южно-Сахалинск,NaN,проспект Мира,NaN,231,NaN,False
3033,4546.0,496.5,KARLA MARKSA 36A ERMAKOVSKOE,"улица Карла Маркса, 36А, село Ермаковское, Ерм...","Россия, Краснодарский край, Динской район, Нов...","Россия, Красноярский край, Ермаковский сельсов...",39.075143,92.405408,45.240610,53.278013,...,Новотитаровское сельское поселение,Ермаковский сельсовет,хутор Карла Маркса,село Ермаковское,NaN,улица Карла Маркса,NaN,36А,NaN,False
3034,152.0,1022.0,ABB 14A MIRA PR-KT ELABUGA,"проспект Мира, 14А, Елабуга, муниципальное обр...","Россия, Московская область, городской округ Пу...","Россия, Республика Татарстан (Татарстан), Елаб...",37.754413,52.016479,56.192430,55.754152,...,городской округ Пушкинский,муниципальное образование Елабуга,NaN,Елабуга,NaN,проспект Мира,NaN,14А,NaN,False
3035,7395.0,5478.0,"D. 2B, UL. MIRA SOCHI G","улица Мира, 4, Адлерский район, Сочи, Краснода...",Россия,"Россия, Краснодарский край, городской округ Со...",99.505405,40.000659,61.698657,43.406042,...,NaN,городской округ Сочи,NaN,село Весёлое,NaN,улица Мира,NaN,2/2,NaN,False


In [ ]:
def apply_rule(df):
    """
    Функция объединяет адреса при совпадении верхних компонентов (страна, регион, район, населённый пункт) между RU и EN.

    Алгоритм работы:
    1. Выполняется нормализация текстовых полей для обеих версий адреса (RU и EN), включая страну, регион, район, населённый пункт, улицу и дом.  
    2. Проверяется, совпадают ли верхние компоненты (country, province, area, locality).  
    3. Если совпадают — функция анализирует наличие улицы и дома:
       - допускается, что улица или дом могут присутствовать только с одной стороны (RU или EN);
       - при необходимости недостающие значения дополняются из второй версии.
    4. Для таких строк создаётся итоговый адрес `geo_address`, объединяющий:
       - верхние компоненты из EN (т.к. они обычно стабильнее для геокодера);
       - наиболее полные значения `street` и `house` (берутся из RU, если там есть данные, иначе из EN).
    5. Итог формируется в виде DataFrame с финальными колонками (`geo_address`, `geo_lon`, `geo_lat`, `country`, `province`, `area`, `locality`, `street`, `house` и др.)  
       Остальные строки, где правило не выполнено, возвращаются отдельно для последующих этапов очистки или ручной обработки.
    """
    # Нормализованные верхние компоненты
    country_en, country_ru = df["country_en"].map(norm), df["country_ru"].map(norm)
    province_en, province_ru = df["province_en"].map(norm), df["province_ru"].map(norm)
    area_en, area_ru = df["area_en"].map(norm), df["area_ru"].map(norm)
    locality_en, locality_ru = df["locality_en"].map(norm), df["locality_ru"].map(norm)

    # Нормализованные улица/дом
    street_en_, street_ru_ = df["street_en"].map(norm), df["street_ru"].map(norm)
    house_ru_, house_ru_ = df["house_en"].map(norm), df["house_ru"].map(norm)

    # Совпадение верхних компонентов
    components_equal = (country_en == country_ru) & (province_en == province_ru) & (area_en == area_ru) & (locality_en == locality_ru)

    # Логика наличия улицы/дома
    street_any_present = (street_en_ != "") | (street_ru_ != "")
    house_both_empty = (house_ru_ == "") & (house_ru_ == "")
    house_only_en = (house_ru_ != "") & (house_ru_ == "")
    house_only_ru = (house_ru_ == "") & (house_ru_ != "")
    street_only_en = (street_en_ != "") & (street_ru_ == "")
    street_only_ru = (street_en_ == "") & (street_ru_ != "")

    # Итоговая маска rule1
    rule_mask = components_equal & ((street_any_present & (house_both_empty | house_only_en | house_only_ru)) | 
                                     ((street_only_ru & house_only_en) | (street_only_en & house_only_ru)))

    # Индексы подходящих
    idx = df.index[rule_mask]

    # Готовим финальные версии улицы/дома
    street_final = df["street_en"].copy()
    house_final  = df["house_en"].copy()

    # Для каждой строки подтягиваем недостающие из RU при необходимости
    for i in idx:
        street_ru, street_en = df.at[i, "street_ru"], df.at[i, "street_en"]
        house_ru, house_en = df.at[i, "house_ru"],  df.at[i, "house_en"]
        street_final.at[i] = street_ru if norm(street_ru) else street_en
        house_final.at[i]  = house_ru if norm(house_ru) else house_en

    def _assemble_address(country, province, area, locality, street, house) -> object:
        """
        Формирует полный адрес (geo_address) из отдельных компонентов с предварительной нормализацией. 
        Использует только непустые значения — страна, регион, район, населённый пункт, улица и дом (если указан). 
        Возвращает собранную строку адреса или NaN, если все компоненты отсутствуют.
        """
        # Собираем только непустые компоненты
        parts = [part for part in [country, province, area, locality, street] if norm(part)]

        # Добавляем дом, если есть
        if norm(house):
            parts.append(str(house))

        # Возвращаем строку или NaN
        return ", ".join(parts) if parts else np.nan
    
    # Сборка финального geo_address для rule1
    final_geo = []
    for i in df.index:
        if i in idx:
            final_geo.append(_assemble_address(df.at[i,"country_en"], df.at[i,"province_en"], 
                                               df.at[i,"area_en"], df.at[i,"locality_en"], 
                                               street_final.at[i], house_final.at[i]))
        else:
            final_geo.append(np.nan)

    # Составляем итоговый датафрейм по схеме финальных колонок
    df_rule = df.loc[idx, ["id", "atm_group", "address", "geo_address_en", "geo_lon_en", "geo_lat_en", "country_en",
                           "province_en", "area_en", "locality_en", "street_en", "house_en", "target", "isTrain"]].rename(columns={
                               "geo_address_en": "geo_address", "geo_lon_en": "geo_lon", "geo_lat_en": "geo_lat", 
                               "country_en": "country", "province_en": "province", "area_en": "area", "locality_en": "locality",
                               "street_en":" street", "house_en": "house"}).copy()

    # Подменяем street/house финальными
    df_rule.loc[:, "street"] = street_final.loc[idx].values
    df_rule.loc[:, "house"]  = house_final.loc[idx].values
    
    # Подставляем собранный geo_address
    df_rule.loc[:, "geo_address"] = pd.Series(final_geo, index=df.index).loc[idx].values

    return df_rule[["id","atm_group","address","geo_address","geo_lon","geo_lat", 
                    "country","province","area","locality","street","house","target","isTrain"]], df.loc[~rule_mask].reset_index(drop=True)

In [ ]:
# Загружаем уже очищенные данные
cleaned_data = pd.read_csv("data/cleaned_data.csv", encoding="utf-8")

# Применяем правило склейки улицы/дома
rule_accepted, df_left_to_clean = apply_rule(df_left_to_clean)

# Объединияем принятые с очищенными
cleaned_data = pd.concat([cleaned_data, rule_accepted], ignore_index=True)

# Сохраняем обновлённые очищенные данные
cleaned_data.to_csv("data/cleaned_data.csv", index=False, encoding="utf-8")

# Выводим оставшиеся строки после обработки
display(df_left_to_clean)

,id,atm_group,address,address_rus,geo_address_en,geo_address_ru,geo_lon_en,geo_lon_ru,geo_lat_en,geo_lat_ru,...,area_en,area_ru,locality_en,locality_ru,street_en,street_ru,house_en,house_ru,target,isTrain
1,4489.0,496.5,LENINA 56 AKSHA,"улица Ленина, 52/7, село Акша, Акшинский район...","Россия, Забайкальский край, Акшинский муниципа...","Россия, Забайкальский край, Акшинский муниципа...",113.286576,113.286909,50.280643,50.281236,...,Акшинский муниципальный округ,Акшинский муниципальный округ,село Акша,село Акша,улица Ленина,улица Ленина,54,52/7,-0.059237,True
3,4606.0,496.5,KLUCHEVSKAYA 6B ULAN-UDE,"Ключевская улица, 6, Улан-Удэ, Республика Буря...","Россия, Республика Бурятия, Улан-Удэ, Ключевск...","Россия, Республика Бурятия, Улан-Удэ, Ключевск...",107.632876,107.632517,51.820955,51.820888,...,городской округ Улан-Удэ,городской округ Улан-Удэ,Улан-Удэ,Улан-Удэ,Ключевская улица,Ключевская улица,6Б,6,0.040712,True
6,8566.0,496.5,TEKSTILSCHIKOV 18 LENINSK-KUZN,"проспект Текстильщиков, 18, Ленинск-Кузнецкий,...","Россия, Москва, 8-я улица Текстильщиков, 18","Россия, Кемеровская область — Кузбасс, Ленинск...",37.747675,86.213438,55.701354,54.697469,...,NaN,Ленинск-Кузнецкий муниципальный округ,Москва,Ленинск-Кузнецкий,8-я улица Текстильщиков,проспект Текстильщиков,18,18,-0.059520,True
7,8594.0,496.5,"MIRA AVE, 27K/1 KUVANDYK","проспект Мира, 27, Кувандык, Оренбургская обла...","Россия, Оренбургская область, Кувандык, проспе...","Россия, Оренбургская область, Кувандык, проспе...",57.376143,57.376188,51.481821,51.482219,...,Кувандыкский муниципальный округ,Кувандыкский муниципальный округ,Кувандык,Кувандык,проспект Мира,проспект Мира,27К/1,27,-0.090312,True
8,8601.0,496.5,POBEDI 31B LESOSIBIRSK,"улица Победы, 31, Лесосибирск, Красноярский кр...","Россия, Красноярский край, Лесосибирск, улица ...","Россия, Красноярский край, Лесосибирск, улица ...",92.531828,92.531119,58.185688,58.186134,...,городской округ Лесосибирск,городской округ Лесосибирск,Лесосибирск,Лесосибирск,улица Победы,улица Победы,31Б,31,0.043808,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2974,935.0,1942.0,VL.4 NOSOVIKHINSKOE HW. MKR.SALTYKOV,"Носовихинское шоссе, вл4, микрорайон Салтыковк...","Россия, Свердловская область, Кировградский му...","Россия, Московская область, Балашиха, микрорай...",59.958233,37.888522,57.449475,55.749674,...,Кировградский муниципальный округ,городской округ Балашиха,NaN,Балашиха,NaN,Носовихинское шоссе,NaN,вл4,NaN,False
2975,2948.0,496.5,MIRA 231 YUZHNO-SAKHA,"проспект Мира, 231, Южно-Сахалинск, Сахалинска...","Россия, посёлок городского типа Новоайдар, ква...","Россия, Сахалинская область, Южно-Сахалинск, п...",38.992588,142.744517,48.964270,46.943684,...,муниципальный округ Новоайдарский,городской округ Южно-Сахалинск,посёлок городского типа Новоайдар,Южно-Сахалинск,NaN,проспект Мира,NaN,231,NaN,False
2976,4546.0,496.5,KARLA MARKSA 36A ERMAKOVSKOE,"улица Карла Маркса, 36А, село Ермаковское, Ерм...","Россия, Краснодарский край, Динской район, Нов...","Россия, Красноярский край, Ермаковский сельсов...",39.075143,92.405408,45.240610,53.278013,...,Новотитаровское сельское поселение,Ермаковский сельсовет,хутор Карла Маркса,село Ермаковское,NaN,улица Карла Маркса,NaN,36А,NaN,False
2978,7395.0,5478.0,"D. 2B, UL. MIRA SOCHI G","улица Мира, 4, Адлерский район, Сочи, Краснода...",Россия,"Россия, Краснодарский край, городской округ Со...",99.505405,40.000659,61.698657,43.406042,...,NaN,городской округ Сочи,NaN,село Весёлое,NaN,улица Мира,NaN,2/2,NaN,False


In [ ]:
# Путь для сохранения датасета
OUTPUT_PATH = "data/all_data_geocoding_results_updated.csv"

# В словаре updates лежат строки, которые не вышло обработать кодом по различным причинам (напр. нет русской транслитерации и геокодер не смог получить адрес, неполные/неверные адреса и т.д.) и они потребовали ручной проверки и доработки:
updates = {6.0: "проспект Хо Ши Мина, 19, Ульяновск",
 199.0: "улица Мира, 41/1, Пермь",
 338.0: "Краснококшайская улица, 158, Казань, Республика Татарстан (Татарстан)",
 347.0: "Набережночелнинский проспект, 10Б, Набережные Челны, Республика Татарстан (Татарстан)",
 410.0: "проспект Красных Партизан, 53, Усолье-Сибирское, Иркутская область",
 556.0: "Можайское шоссе, 133А, Одинцово, Московская область",
 560.0: "Наро-Фоминское шоссе, с10, Кубинка, Одинцовский городской округ, Московская область",
 568.0: "Измайловский проспект, вл54, Москва",
 586.0: "улица Борисовские Пруды, 26, Москва",
 705.0: "бульвар Яна Райниса, 45к1, Москва",
 706.0: "бульвар Яна Райниса, 45к1, Москва",
 707.0: "бульвар Яна Райниса, 45к1, Москва",
 708.0: "бульвар Яна Райниса, 45к1, Москва",
 727.0: "Московский проспект, 74, Санкт-Петербург",
 774.0: "улица Черняховского, 30, Калининград",
 775.0: "улица Черняховского, 34, Калининград",
 918.0: "Носовихинское шоссе, 7, Реутов, Московская область",
 957.0: "улица Большая Ордынка, 63, Москва",
 989.0: "Нижняя Красносельская улица, 45/17, Москва",
 990.0: "Нижняя Красносельская улица, 45/17, Москва",
 991.0: "Нижняя Красносельская улица, 45/17, Москва",
 992.0: "Нижняя Красносельская улица, 45/17, Москва",
 1011.0: "Ходынский бульвар, 4, Москва",
 1028.0: "Солнечная улица, 18/3, Краснодар",
 1156.0: "улица Ленина, 34, Курган",
 1164.0: "Северная улица, 6, микрорайон Центральный, Сочи, Краснодарский край",
 1260.0: "улица 30 лет Победы, 46, Сургут, Ханты-Мансийский автономный округ — Югра",
 1273.0: "улица 22-го Партсъезда, 4, Самара",
 1296.0: "21-я Амурская улица, 21, Омск",
 1305.0: "улица Федюнинского, 55, Тюмень",
 1323.0: "Рабоче-Крестьянская улица, 9Б, Волгоград",
 1338.0: "бульвар Строителей, 28, Кемерово",
 1356.0: "Варшавское шоссе, 143А, Москва",
 1380.0: "Ульяновское шоссе, 38, Сызрань, Самарская область",
 1384.0: "Большая Семёновская улица, 16, Москва",
 1389.0: "проспект Ленина, 217с2, Томск",
 1469.0: "Гусинобродское шоссе, 20, Новосибирск",
 1497.0: "улица Щорса, 64, микрорайон Пригородный, посёлок Дубовое, Дубовское сельское поселение, Белгородский район",
 1572.0: "Большая Горная улица, 157, Саратов",
 1573.0: "проспект Строителей, 1к3А, Саратов",
 1577.0: "Большая Садовая улица, 153/163, Саратов",
 1591.0: "Улица Бережковская набережная 16а Москва",
 1651.0: "бульвар Новаторов, 102, Санкт-Петербург",
 1680.0: "бульвар Новаторов, 8, Санкт-Петербург",
 1712.0: "проспект имени Газеты Красноярский Рабочий, 199, Красноярск",
 1789.0: "Московский проспект, 193, Санкт-Петербург",
 1829.0: "Московский проспект, 74, Санкт-Петербург",
 1870.0: "Торговый проезд, 1, Саратов",
 1886.0: "проспект имени 50 лет Октября, 91Д, Саратов",
 1964.0: "Красноармейская улица, 111, Йошкар-Ола, Республика Марий Эл",
 2041.0: "Алтуфьевское шоссе, 70к1, Москва",
 2152.0: "улица Ползунова, 15, Новосибирск",
 2164.0: "ул. Сибиряков-Гвардейцев, 54/3, Новосибирск",
 2208.0: "Интернациональная улица, 43, Омск",
 2247.0: "Телевизорная улица, 1с4, Красноярск",
 2475.0: "Радиотехническая улица, 1Ак2, Елец, Липецкая область",
 2478.0: "улица Белгородского Полка, 34, Белгород",
 2491.0: "Пресненская набережная, 12, Москва",
 2529.0: "Сельскохозяйственная улица, 15/1, Москва",
 2557.0: "Комсомольская площадь, 2, Москва",
 2564.0: "Пресненская набережная, 12, Москва",
 2566.0: "площадь Киевского Вокзала, 2, Москва",
 2567.0: "Староватутинский проезд, 14, Москва",
 2569.0: "улица Ленинская Слобода, 26с5, Москва",
 2570.0: "улица Сергея Макеева, 13, Москва",
 2575.0: "Артиллерийская улица, 100В, Челябинск",
 2608.0: "улица 25 Сентября, 35А, Смоленск",
 2615.0: "улица 25 Сентября, 35А, Смоленск",
 2658.0: "улица Щорса, 8В, Белгород",
 2662.0: "улица Королёва, 9А, Белгород",
 2664.0: "улица Пугачёва, 5, Белгород",
 2725.0: "Павловский тракт, 251В, Барнаул, Алтайский край",
 2746.0: "Российская улица, 6, микрорайон Крутая Горка, Омск",
 2776.0: "Железнодорожная улица, 8, Октябрьский район, городской округ Красноярск",
 2834.0: "Кавалерийская улица, 2, Новосибирск",
 2906.0: "улица Геолога Кудрявого, 24, посёлок городского типа Хандыга, Томпонский район, Республика Саха (Якутия)",
 2989.0: "2-й Сыромятнический переулок, 1, Москва",
 3107.0: "Красноармейская улица, 63/90, Ростов-на-Дону",
 3173.0: "Преображенская площадь, 8, Москва",
 3261.0: "Старопетровский проезд, 11к2, Москва",
 3284.0: "2-я Комсомольская улица, 19, планировочный район Ново-Александровск, Южно-Сахалинск, Сахалинская область",
 3293.0: "1-й Красногвардейский проезд, 15, Москва",
 3326.0: "улица Кржижановского, 24/35к6, Москва",
 3333.0: "Октябрьский проспект, 105, Тверь",
 3348.0: "улица Черняховского, 76, Калининград",
 3356.0: "микрорайон Интернациональный, 7А, Саяногорск, Республика Хакасия",
 3400.0: "Павелецкая площадь, 3, Москва",
 3426.0: "улица Партизана Железняка, 1, Красноярск",
 3506.0: "улица Героев-Тихоокеанцев, 5А, Владивосток, Приморский край",
 3522.0: "улица Маршала Полубоярова, 16к1, Москва",
 3535.0: "Авиационная улица, 12, территория Комсомольск 31, Комсомольский район, Хабаровский край",
 3560.0: "улица Суханова, 55Б, Уссурийск, Приморский край",
 3612.0: "улица 78-й Добровольческой Бригады, 11, Красноярск",
 3700.0: "проспект Кораблестроителей, 22Б, Нижний Новгород",
 3736.0: "бульвар Комарова, 24А, микрорайон Северный, Ростов-на-Дону",
 3756.0: "2-й Южнопортовый проезд, 18с9, Москва",
 3779.0: "Кронштадтский бульвар, 37Б, Москва",
 3968.0: "Ленинградская улица, 37А, Хабаровск",
 3996.0: "Зеленоград, к601, Москва",
 4034.0: "метро Площадь Ленина Новосибирск",
 4038.0: "Кронштадтская улица, 113, Вязьма, Смоленская область",
 4137.0: "улица Володи Головатого, 313, Краснодар",
 4209.0: "5-й микрорайон, 29, Чита, Забайкальский край",
 4316.0: "Краснодарский край, Копанской хутор, 85",
 4412.0: "Алтуфьевское шоссе, 22, Москва",
 4427.0: "Ленинская улица, 37, село Первомайское, Томская область",
 4741.0: "улица Братьев Кашириных, 133, Челябинск",
 4782.0: "улица Героев Сталинградской Битвы, 35В, Йошкар-Ола, Республика Марий Эл",
 4787.0: "Коммунистический проспект, 33, Южно-Сахалинск, Сахалинская область",
 4796.0: "улица Курчатова, 15, Красноярск",
 4805.0: "улица Приборостроителей, 3А, Рыбинск, Ярославская область",
 4868.0: "Московское шоссе, 30В, Нижний Новгород",
 4874.0: "улица 78-й Добровольческой Бригады, 12, Красноярск",
 4877.0: "Красноармейское шоссе, 23, Вязьма, Смоленская область",
 5019.0: "Комсомольский проспект, 28Д, Челябинск",
 5021.0: "проспект Обводный канал, 91, Архангельск",
 5047.0: "Серебряническая набережная, 29, Москва",
 5078.0: "улица Братьев Кашириных, 99А, Челябинск",
 5146.0: "Байкальская улица, 126/1, Иркутск",
 5148.0: "микрорайон Первомайский, 54, Иркутск",
 5181.0: "Светлогорская улица, 15А, Пермь",
 5183.0: "Октябрьская улица, 80А, Курск",
 5267.0: "10-я Красноармейская улица, 22, Санкт-Петербург",
 5277.0: "микрорайон Юбилейный, 13/1, Иркутск",
 5284.0: "улица Фаворского, 12, Санкт-Петербург",
 5304.0: "улица 50-й Армии, 1, Брянск",
 5338.0: "Коммунистическая улица, 105, Вольск, Саратовская область",
 5347.0: "улица Богдана Хмельницкого, 109к3, Чебоксары, Чувашская Республика — Чувашия",
 5354.0: "улица 78-й Добровольческой Бригады, 15, Красноярск",
 5407.0: "Астраханская улица, 189А, Тамбов",
 5464.0: "Большая Морская улица, 30, Санкт-Петербург",
 5513.0: "проспект Машиностроителей, 22, Ярославль",
 5552.0: "Севастопольская улица, 101А, Губкин, Белгородская область",
 5581.0: "Интернациональная улица, 79, Усолье-Сибирское, Иркутская область",
 5586.0: "проспект Тракторостроителей, 76, Чебоксары, Чувашская Республика — Чувашия",
 5639.0: "Красноармейская улица, 123, Благовещенск, Амурская область",
 5647.0: "проспект Зои Космодемьянской, 88, Азов, Ростовская область",
 5648.0: "проспект Зои Космодемьянской, 88, Азов, Ростовская область",
 5685.0: "проспект Красных Партизан, 24, Усолье-Сибирское, Иркутская область",
 5686.0: "проспект Красных Партизан, 24, Усолье-Сибирское, Иркутская область",
 5687.0: "проспект Красных Партизан, 24, Усолье-Сибирское, Иркутская область",
 5699.0: "проспект Зои Космодемьянской, 88, Азов, Ростовская область",
 5846.0: "Красноармейская улица, 123, Благовещенск, Амурская область",
 5877.0: "Коммунистическая улица, 47А, Улан-Удэ, Республика Бурятия",
 5878.0: "Коммунистическая улица, 47А, Улан-Удэ, Республика Бурятия",
 5923.0: "проспект Советской Армии, 55, Новокузнецк, Кемеровская область — Кузбасс",
 5980.0: "Красноармейский проспект, 47А, Барнаул, Алтайский край",
 6050.0: "проспект Мусы Джалиля, 25, Набережные Челны, Республика Татарстан (Татарстан)",
 6074.0: "площадь Мира, 7, Таганрог, Ростовская область",
 6075.0: "Коммунистическая улица, 47А, Улан-Удэ, Республика Бурятия",
 6076.0: "Коммунистическая улица, 47А, Улан-Удэ, Республика Бурятия",
 6100.0: "улица Октябрьской Революции, 9к1, Смоленск",
 6133.0: "Краснознамённая улица, 23А, Краснознаменск, Московская область",
 6155.0: "улица имени Генерала Меркулова, 14, Липецк",
 6271.0: "Московский проспект, 220, Санкт-Петербург",
 6339.0: "проспект Братьев Коростелёвых, 22, Оренбург",
 6363.0: "проспект Богдана Хмельницкого, 137Т, Белгород",
 6404.0: "бульвар Свободы, 2, Сургут, Ханты-Мансийский автономный округ — Югра",
 6464.0: "проспект Кораблестроителей, 22Б, Нижний Новгород",
 6524.0: "Революционная улица, 11Б, Тольятти, Самарская область",
 6525.0: "Революционная улица, 11Б, Тольятти, Самарская область",
 6526.0: "Революционная улица, 11Б, Тольятти, Самарская область",
 6527.0: "Революционная улица, 11Б, Тольятти, Самарская область",
 6548.0: "Московское шоссе, 4Ас1, Самара",
 6576.0: "улица Максима Горького, 262, Нижний Новгород",
 6629.0: "Академическая улица, 31, Иркутск",
 6631.0: "улица Максима Горького, 152, Нижний Новгород",
 6632.0: "улица Максима Горького, 152, Нижний Новгород",
 6633.0: "улица Максима Горького, 152, Нижний Новгород",
 6634.0: "улица Максима Горького, 152, Нижний Новгород",
 6662.0: "Академическая улица, 12, Иркутск",
 6688.0: "бульвар Энтузиастов, 2, Москва",
 6689.0: "бульвар Энтузиастов, 2, Москва",
 6690.0: "бульвар Энтузиастов, 2, Москва",
 6691.0: "бульвар Энтузиастов, 2, Москва",
 6738.0: "улица Луначарского, 128А, Карпинск, Свердловская область",
 6739.0: "улица Луначарского, 128А, Карпинск, Свердловская область",
 6911.0: "Белореченская улица, 15, Екатеринбург, Свердловская область",
 6912.0: "Белореченская улица, 15, Екатеринбург, Свердловская область",
 7037.0: "набережная Обводного канала, 120, Санкт-Петербург",
 7046.0: "улица Скворцова-Степанова, 7, Тверь",
 7120.0: "Байкальская улица, 250/1, Иркутск",
 7128.0: "Байкальская улица, 250/1, Иркутск",
 7145.0: "улица Бабушкина, 10А, Санкт-Петербург",
 7149.0: "Гостилицкое шоссе, 58, Петергоф, Санкт-Петербург",
 7174.0: "Шереметьевская улица, 9, Рязань",
 7175.0: "Ушаковская набережная, 17/1, Санкт-Петербург",
 7232.0: "улица Станкостроителей, 14, Ульяновск",
 7250.0: "Вокзальная площадь, 3, Калуга",
 7254.0: "улица Корабельная Набережная, 2, Владивосток, Приморский край",
 7275.0: "улица Костюшко-Григоровича, 5, Чита, Забайкальский край",
 7303.0: "Красноармейская улица, 116/2, микрорайон Центральный, Краснодар",
 7314.0: "Тверской бульвар, 15с1, Москва",
 7343.0: "микрорайон Юбилейный, 19/1, Иркутск",
 7410.0: "Благовещенская улица, 4, Вологда",
 7443.0: "Ульяновский проспект, 14, Ульяновск",
 7507.0: "площадь Широкова, 1, Центральный район, городской округ Кострома",
 7522.0: "улица Чернышевского, 82, Уфа, Республика Башкортостан",
 7545.0: "проспект Победа Революции, 113, Шахты, Ростовская область",
 7565.0: "улица Энергетиков, 21Б, Челябинск",
 7572.0: "проспект Ленина, 56, Ростов-на-Дону",
 7763.0: "1-я Станционная улица, 25, Искитим, Новосибирская область",
 7881.0: "улица Октябрьской Революции, 5, Туапсе, Краснодарский край",
 7899.0: "улица Октябрьской Революции, 5, Туапсе, Краснодарский край",
 7947.0: "улица Собинова, 7Е, Пенза",
 7973.0: "проспект Ленина, 56, Ростов-на-Дону",
 7988.0: "улица Собинова, 7Е, Пенза",
 8026.0: "улица Октябрьской Революции, 9к1, Смоленск",
 8035.0: "микрорайон Первомайский, 23, Иркутск",
 8147.0: "улица Карбышева, 9А, Санкт-Петербург",
 8172.0: "Выборгское шоссе, 11А, территория Возрожденская, Каменногорское городское поселение, Выборгский район, Ленинградская область",
 8174.0: "Центральная улица, 13А, Нариманов, Астраханская область",
 8244.0: "Новоторжская улица, 12А, Тверь",
 8245.0: "Новоторжская улица, 12А, Тверь",
 8246.0: "Новоторжская улица, 12А, Тверь",
 8324.0: "Интернациональная улица, 16А, Тамбов",
 8325.0: "Интернациональная улица, 16А, Тамбов",
 8402.0: "Московский проспект, 220, Санкт-Петербург",
 8413.0: "Комендантская площадь, 1, Санкт-Петербург",
 8495.0: "Советская улица, 25, село Майя, Мегино-Кангаласский улус, Республика Саха (Якутия)",
 157.0: "Красноармейская улица, 42, Йошкар-Ола, Республика Марий Эл",
 1930.0: "проспект Строителей, 37, Альметьевск, Республика Татарстан (Татарстан)",
 2896.0: "проспект Максима Горького, 51, Чебоксары, Чувашская Республика — Чувашия",
 4632.0: "улица Салиха Сайдашева, 1А, Бавлы, Республика Татарстан (Татарстан)",
 2025.0: "проспект Строителей, 35, Альметьевск, Республика Татарстан (Татарстан)",
 4497.0: "Окружное шоссе, 37А, Елабуга, Республика Татарстан (Татарстан)",
 295.0: "Окружное шоссе, 37А, Елабуга, Республика Татарстан (Татарстан)",
 1914.0: "Красноармейская улица, 91, Йошкар-Ола, Республика Марий Эл",
 327.0: "улица Мулланура Вахитова, 4, Казань, Республика Татарстан (Татарстан)",
 2014.0: "Окружное шоссе, 7А, Елабуга, Республика Татарстан (Татарстан)",
 2882.0: "улица Митрофана Седина, 176, микрорайон Центральный, Краснодар",
 1907.0: "улица Ленина, 126, Альметьевск, Республика Татарстан (Татарстан)",
 1946.0: "улица А. Абязова, 3, село Старое Дрожжаное, Дрожжановский район, Республика Татарстан (Татарстан)",
 1948.0: "Первомайская улица, 115, Йошкар-Ола, Республика Марий Эл",
 7087.0: "бульвар Комарова, 24А, микрорайон Северный, Ростов-на-Дону",
 326.0: "проспект Победы, 18Б, Казань, Республика Татарстан",
 155.0: "улица Марджани, 34, Елабуга, Республика Татарстан (Татарстан)",
 587.0: "Бумажный проезд, 14с2, Москва",
 7249.0: "Ленинградский проспект, 28В, Кемерово", 
 3126.0: "2-я Хуторская улица, 38Ас9, Москва",
 286.0: "улица Ленина, 88, Пермь",
 5279.0: "улица Меркурьева, 3, посёлок Парголово, Санкт-Петербург",
 7472.0: "Комсомольская улица, 24, Ленинский район, Киров",
 402.0: "Комендантский проспект, 33к1, Санкт-Петербург", 
 8262.0: "улица 70 лет Октября, 25к3, Омск", 
 4487.0: "улица 50-летия Белгородской области, 11, Белгород", 
 7789.0: "улица Забабонова, 19, Болотное, Новосибирская область", 
 1170.0: "Севастопольский проспект, 11Е, Москва",
 8454.0: "улица Ижорского Батальона, 13, Колпино, Санкт-Петербург",
 6065.0: "Молодогвардейская улица, 204, Самара",
 6071.0: "Молодогвардейская улица, 204, Самара",
 5106.0: "Большая Покровская улица, 56, Нижний Новгород",
 5827.0: "Московское шоссе, 16, посёлок Шушары, Пушкинский район, Санкт-Петербург",
 5880.0: "улица Муравьёва-Амурского, 18, Хабаровск",
 5881.0: "улица Муравьёва-Амурского, 18, Хабаровск",
 5882.0: "улица Муравьёва-Амурского, 18, Хабаровск",
 5883.0: "улица Муравьёва-Амурского, 18, Хабаровск",
 7736.0: "Ленинградская улица, 101, посёлок городского типа Карымское, Забайкальский край",
 4948.0: "улица Генерала Попова, 8, Калуга",
 5345.0: "Рязанская улица, 34, Тула",
 8148.0: "бульвар Трудящихся, 12, Колпино, Санкт-Петербург",
 4893.: "Космический проезд, 3А, Петропавловск-Камчатский, Камчатский край", 
 6101.0: "Советская улица, 212/2, Бийск, Алтайский край", 
 6577.0: "Россия, Калининград, улица Профессора Баранова, 30", 
 8138.0: "улица Тимофея Невежина, 3с10, Курган", 
 8143.0: "Россия, Москва, улица Барклая, 7к1", 
 8145.0: "Россия, Москва, улица Барклая, 7к1", 
 8146.0: "Россия, Москва, улица Барклая, 7к1", 
 3223.0: "Россия, Ярославская область, Тутаев, проспект 50-летия Победы, 9А", 
 5325.0: "площадь 50 лет Октября, 3, Балабаново, Боровский муниципальный округ, Калужская область", 
 7517.0: "5-й микрорайон, 31, Лесосибирск, Красноярский край", 
 7852.0: "Железнодорожная улица, 14, Дедовск, муниципальный округ Истра, Московская область", 
 7983.0: "Россия, Санкт-Петербург, Комендантский проспект, 12к1", 
 8210.0: "Россия, Санкт-Петербург, Комендантский проспект, 12к1", 
 3312.0: "Россия, Волгоград, район Семь Ветров, бульвар 30-летия Победы, 1", 
 3331.0: "Россия, Астрахань, проспект Губернатора Анатолия Гужвина, 10",
 3903.0: "Россия, Волгоград, район Семь Ветров, бульвар 30-летия Победы, 1",   
 43.0: "Россия, Вологда, улица Михаила Поповича, 4А", 
 5567.0: "улица Циолковского, 4Ас1, Кремёнки, городское поселение Кремёнки, Жуковский муниципальный округ, Калужская область", 
 5906.0: "Промышленная улица, 9А, Шарья, Костромская область", 
 565.0: "Россия, Волгоград, улица 51-й Гвардейской Дивизии, 29", 
 4885.0: "посёлок городского типа Горный, 6, Забайкальский край", 
 5061.0: "улица Полушкина Роща, 9с14, Ярославль", 
 7406.0: "Россия, Самарская область, Сызрань, 2-я Железнодорожная улица, 8", 
 7676.0: "улица имени Ф.Э. Дзержинского, 9/11, Саратов", 
 8129.0: "Телевизорная улица, 1с4, Красноярск", 
 3017.0: "Россия, Томская область, Шегарское сельское поселение, село Мельниково, Коммунистическая улица, 37", 
 1740.0: "улица Муравьёва-Амурского, 36, Хабаровск", 
 1741.0: "улица Муравьёва-Амурского, 36, Хабаровск", 
 1742.0: "улица Муравьёва-Амурского, 36, Хабаровск", 
 8453.0: "Офицерская улица, 12Вс1, Тольятти, Самарская область",
 4890.0: "Россия, Кемеровская область — Кузбасс, Новокузнецк, улица Новосёлов, 18/1", 
 5353.0: "Право-Булачная улица, 43/1, Казань, Республика Татарстан (Татарстан)", 
 5471.0: "Варшавское шоссе, 1с6, Москва", 
 5570.0: "Симферопольское шоссе, 4с2, Чехов, Московская область", 
 6465.0: "Россия, Санкт-Петербург, проспект Просвещения, 68к8",
 6838.0: "проспект Максима Горького, 10с1, Чебоксары, Чувашская Республика — Чувашия",
 6443.0: "Большая Морская улица, 11, Санкт-Петербург", 
 3627.0: "Россия, Москва, улица Академика Королёва, 19",  
 646.0: "Россия, Москва, 1-я Тверская-Ямская улица, 32",  
 648.0: "Россия, Москва, 1-я Тверская-Ямская улица, 32", 
 6444.0: "Большая Морская улица, 11, Санкт-Петербург",  
 6948.0: "Московский проспект, 6, Санкт-Петербург",
 6441.0: "Большая Морская улица, 11, Санкт-Петербург", 
 4943.0: "Россия, Иваново, 1-я Меланжевая улица, 5А", 
 6442.0: "Большая Морская улица, 11, Санкт-Петербург", 
 7172.0: "шоссе 50 лет Октября, 2А, Мирный, Республика Саха (Якутия)",  
 649.0: "Россия, Москва, 1-я Тверская-Ямская улица, 32",
 8566.0: "Россия, Кемеровская область — Кузбасс, Ленинск-Кузнецкий, проспект Текстильщиков, 18",
 75.0: "Россия, Красноармейский район, станица Старонижестеблиевская, Советская улица, 82",
 139.0: "Россия, Республика Бурятия, Кабанский район, посёлок городского типа Селенгинск, микрорайон Берёзовый, 3А",
 161.0: "Россия, Пермь, улица Маршала Рыбалко, 84",
 434.0: "Россия, Краснодарский край, Динской район, Нововеличковское сельское поселение, станица Нововеличковская, Красная улица, 55Б", 
 510.0: "Россия, Москва, Шереметьевская улица, 20",  
 544.0: "Россия, Москва, улица Лужники, 24с19",  
 553.0: "Алтуфьевское шоссе, 80, Москва",  
 589.0: "Россия, Москва, проспект Андропова, 18к1",  
 651.0: "Комсомольский проспект, 24с1, Москва",  
 652.0: "Комсомольский проспект, 24с1, Москва",
 653.0: "Комсомольский проспект, 24с1, Москва",
 977.0: "Киевское шоссе, 22-й километр, 4с1, район Солнцево, Западный административный округ, Москва", 
 1824.0: "Россия, Ростов-на-Дону, Большая Садовая улица, 46/30", 
 1862.0: "7-й микрорайон, 12, Радужный, Ханты-Мансийский автономный округ — Югра",  
 840.0: "Россия, Калининград, площадь Победы, 4",  
 859.0: "Россия, Иркутская область, Усолье-Сибирское, проспект Красных Партизан, 20",  
 879.0: "Россия, Нижний Новгород, проспект Ленина, 29",  
 880.0: "Россия, Нижний Новгород, проспект Ленина, 29",  
 905.0: "Будённовский проспект, 51, Ростов-на-Дону",  
 906.0: "Будённовский проспект, 51, Ростов-на-Дону",
 981.0: "улица Большая Полянка, 13с1, Москва", 
 1183.0: "Россия, Сахалинская область, Южно-Сахалинск, 2-я Центральная улица, 1Б", 
 1212.0: "Зелёный проспект, 62, Москва", 
 1250.0: "посёлок Красный Бор, 1, Ярославский муниципальный округ", 
 1524.0: "Россия, Ростов-на-Дону, улица Доватора, 267", 
 2283.0: "улица Серова, 5, Кяхта, Республика Бурятия", 
 2357.0: "проспект Ленина, 6, Нерюнгри, Республика Саха (Якутия)", 
 2358.0: "проспект Ленина, 6, Нерюнгри, Республика Саха (Якутия)", 
 2477.0: "улица 60 лет Октября, 1, 10-й микрорайон, Белгород", 
 2539.0: "Россия, Москва, Хорошёвское шоссе, 38", 
 2540.0: "Россия, Москва, Хорошёвское шоссе, 38", 
 2541.0: "Россия, Москва, Хорошёвское шоссе, 38", 
 2542.0: "Россия, Москва, Хорошёвское шоссе, 38", 
 2543.0: "Россия, Москва, Хорошёвское шоссе, 38",
 2544.0: "Россия, Москва, Хорошёвское шоссе, 38", 
 2545.0: "Россия, Москва, Хорошёвское шоссе, 38", 
 2603.0: "Россия, Тюмень, улица Республики, 86к1", 
 2618.0: "улица Лётчика Пилютова, 11к4, Санкт-Петербург", 
 8421.0: "Россия, Пенза, улица Мира, 41", 
 8458.0: "Россия, Забайкальский край, Читинский район, посёлок городского типа Атамановка, Заводская улица, 1", 
 4737.0: "Россия, Московская область, муниципальный округ Истра, село Павловская Слобода, улица 1 Мая", 
 4813.0: "Светлая улица, 33, посёлок Уташ, муниципальный округ Анапа, Краснодарский край", 
 4886.0: "Россия, Забайкальский край, Читинский район, село Домна, улица Труда, 12", 
 4913.0: "Россия, Челябинская область, Сосновский муниципальный округ, посёлок Полетаево, Пионерская улица, 7А", 
 5066.0: "Россия, Республика Башкортостан, Уфа, Рубежная улица, 174", 
 5083.0: "Россия, Нижний Новгород, жилой район Северный, улица Бурденко, 33",
 5180.0: "улица Металлургов, 80Г, Тула", 
 5184.0: "Россия, Краснодарский край, Сочи, жилой район Адлер, улица Голубые Дали, 13А", 
 5209.0: "улица Кузнецовой, 14, посёлок Молодёжный, Иркутский муниципальный округ", 
 5469.0: "улица Кирова, 95, Павловский Посад, Московская область", 
 5517.0: "Россия, Нижний Новгород, улица Плотникова, 3", 
 5609.0: "Россия, Амурская область, Благовещенск, улица Чехова, 50",
 5610.0: "Россия, Амурская область, Тында, Новая улица, 4", 
 5770.0: "Россия, Чувашская Республика — Чувашия, городской округ Чебоксары, посёлок городского типа Новые Лапсары",
 5884.0: "Россия, Пенза, улица Карпинского, 37А", 
 5903.0: "Россия, Липецкая область, Елец, улица Орджоникидзе, 6", 
 5974.0: "Россия, Ростовская область, посёлок городского типа Усть-Донецкий, улица Горького, 69А", 
 6029.0: "Россия, Тульская область, Венёв, улица Бундурина, 7", 
 6033.0: "Россия, Тула, проспект Ленина, 54", 
 6191.0: "Ленинский проспект, 174П, Воронеж", 
 6626.0: "Ленинский проспект, 174П, Воронеж", 
 6709.0: "Россия, Волгоградская область, Волжский, улица Мира, 74", 
 6968.0: "Россия, Свердловская область, Талицкий муниципальный округ, посёлок Троицкий, улица Мира, 11", 
 6993.0: "Россия, Московская область, Клин, улица Маяковского, 10", 
 7038.0: "Россия, Амурская область, Свободный, улица Карла Маркса, 20", 
 7114.0: "Россия, Тула, улица Фрунзе, 8", 
 7355.0: "Россия, Алтайский край, село Кулунда, улица Лермонтова, 2/9",
 4489.0: "Россия, Забайкальский край, Акшинский муниципальный округ, село Акша, улица Ленина, 54", 
 4606.0: "Россия, Республика Бурятия, Улан-Удэ, Ключевская улица, 6Б", 
 8594.0: "Россия, Оренбургская область, Кувандык, проспект Мира, 27К/1", 
 8601.0: "Россия, Красноярский край, Лесосибирск, улица Победы, 31Б", 
 8709.0: "Россия, Республика Адыгея (Адыгея), Майкоп, Краснооктябрьская улица, 66В", 
 8722.0: "Россия, Республика Бурятия, Улан-Удэ, проспект Автомобилистов, 4А", 
 1901.0: "набережная Чёрной речки, 47с1, Санкт-Петербург",  
 470.0: "Милицейская улица, 8В, Южно-Сахалинск, Сахалинская область", 
 1266.0: "Россия, Томск, улица Вершинина, 76",
 1618.0: "Россия, Санкт-Петербург, Невский проспект, 35А", 
 2186.0: "Россия, Омск, 6-я Станционная улица, 2к2", 
 2284.0: "Россия, Иркутская область, Ангарск, 192-й квартал, 12", 
 2416.0: "Россия, Приморский край, Владивосток, Шилкинская улица, 42", 
 2712.0: "Россия, Алтайский край, Барнаул, улица Островского, 34Б", 
 8420.0: "Россия, Пенза, Окружная улица, 27В", 
 8425.0: "Россия, Забайкальский край, Чита, Деповская улица, 1", 
 8433.0: "Россия, Алтайский край, городской округ Бийск, посёлок Нагорный, Туркменская улица, 3/1",
 4731.0: "Россия, Алтайский край, Барнаул, проспект Калинина, 16В", 
 4912.0: "Россия, Московская область, Сергиево-Посадский городской округ, Хотьково, улица Михеенко, 14", 
 4983.0: "Россия, Белгород, бульвар Юности, 5Б", 
 5149.0: "Россия, Вологда, Сухонская улица, 1Б", 
 5306.0: "Россия, Удмуртская Республика, Ижевск, улица Гагарина, 49Б", 
 5322.0: "Россия, Хабаровск, Матвеевское шоссе, 28", 
 6748.0: "Россия, Республика Дагестан, Каспийск, Приморская улица, 20", 
 7063.0: "Россия, Хабаровский край, Комсомольск-на-Амуре, Водонасосная улица, 1",
 7094.0: "Россия, Самарская область, Сызрань, улица Смирницкой, 28А",
 7139.0: "Россия, Московская область, Мытищи, Осташковское шоссе, 1", 
 7341.0: "Россия, Республика Бурятия, Улан-Удэ, улица Борсоева, 19Б", 
 7385.0: "Россия, Астрахань, микрорайон имени Бабаевского, Бульварная улица, 6В", 
 7436.0: "Россия, Приморский край, Владивосток, Ивановская улица, 4", 
 7482.0: "Россия, Кемеровская область — Кузбасс, Анжеро-Судженск, Станционная улица, 11", 
 7559.0: "Россия, Еврейская автономная область, Облучье, Пролетарская улица, 17", 
 7716.0: "Россия, Иркутская область, Саянск, микрорайон Мирный, 2/1",
 7733.0: "Россия, Республика Бурятия, Заиграевский район, городское поселение Онохой, посёлок городского типа Онохой, улица Терешковой, 12А",
 7782.0: "Россия, Липецкая область, Елец, улица Орджоникидзе, 6", 
 7800.0: "Россия, Липецкая область, Елец, улица Орджоникидзе, 6", 
 7812.0: "Россия, Амурская область, Белогорск, улица Кирова, 188", 
 7844.0: "Россия, Тула, Оборонная улица, 85", 
 7873.0: "Россия, городской округ Нижний Новгород, Канавинский район, площадь Революции, 9", 
 7942.0: "Россия, Кострома, Профсоюзная улица, 25Б", 
 8164.0: "Россия, Краснодарский край, Горячий Ключ, улица Революции, 4", 
 8258.0: "Россия, Тула, проспект Ленина, 54",
 8259.0: "Россия, Тула, проспект Ленина, 54", 
 8288.0: "Россия, Курск, улица Дзержинского, 67", 
 8350.0: "Россия, Республика Марий Эл, Волжск, улица Ленина, 52Б", 
 3232.0: "Россия, Кемеровская область — Кузбасс, Киселёвск, Жемчужная улица, 5А", 
 3271.0: "Россия, Приморский край, Владивосток, Ивановская улица, 4", 
 3353.0: "Россия, Кемерово, Грузовая улица, 23", 
 3402.0: "Россия, Тула, улица Фрунзе, 7А", 
 3429.0: "Россия, Владимирская область, Вязники, улица Сенькова, 9/8",
 3542.0: "Россия, Тюмень, улица Мельникайте, 125", 
 3553.0: "Россия, Республика Тыва, Кызылский кожуун, посёлок городского типа Каа-Хем, Шахтёрская улица, 16А", 
 3625.0: "Россия, Приморский край, Пожарский муниципальный округ, посёлок городского типа Лучегорск, 3-й микрорайон, 30", 
 3628.0: "Россия, Санкт-Петербург, Петроградская набережная, 18", 
 3704.0: "Россия, Нижний Новгород, улица Веденяпина, 1Б", 
 3879.0: "Россия, Москва, Нахимовский проспект, 11к1", 
 3900.0: "Россия, Красноярский край, Ачинск, 7-й микрорайон, 13", 
 4012.0: "Россия, Хабаровск, Матвеевское шоссе, 28", 
 4020.0: "проспект Космонавтов, 6В, Барнаул, Алтайский край",
 4065.0: "5-я Северная улица, 193, Омск", 
 4070.0: "Россия, Калужская область, Боровский муниципальный округ, село Ворсино, улица Лыскина, 1", 
 4364.0: "Россия, Белгородская область, Старый Оскол, микрорайон Олимпийский, 56", 
 4433.0: "Россия, Приморский край, Владивосток, улица Корабельная Набережная, 15А", 
 2893.0: "проспект Мира, 124к4, Москва", 
 1980.0: "Россия, Краснодарский край, Лабинск, улица Ленина, 164", 
 2970.0: "Россия, Тула, улица Максима Горького, 14А", 
 4605.0: "Россия, Сахалинская область, Южно-Сахалинск, улица Ленина, 123", 
 4711.0: "Россия, Кировская область, посёлок городского типа Кильмезь, Зелёная улица, 13В",
 8488.0: "Россия, Пензенская область, Кузнецк, Комсомольская улица, 34Б", 
 8501.0: "Россия, Новосибирский район, Станционный сельсовет, посёлок Садовый, Совхозная улица, 7", 
 8520.0: "Россия, Республика Тыва, Монгун-Тайгинский кожуун, село Мугур-Аксы, улица Саны-Шири, 46", 
 8670.0: "Россия, Республика Саха (Якутия), Томпонский район, посёлок городского типа Хандыга, Магаданская улица, 29/4", 
 8693.0: "Россия, Воронежская область, посёлок городского типа Каменка, Полевая улица, 8", 
 8730.0: "Россия, Республика Дагестан, Дербент, улица Гейдара Алиева, 12Б", 
 8761.0: "Россия, Москва, Волгоградский проспект, 121/35", 
 8804.0: "Россия, Республика Калмыкия, Элиста, 3-й микрорайон, 22",   
 26.0: "Россия, Москва, улица Арбат, 1",
 83.0: "Россия, Кировская область, Яранск, улица Кирова, 28",  
 123.0: "Советская улица, 83А, село Дебёсы, Дебёсский район, Удмуртская Республика",  
 194.0: "Россия, Пермский край, Чайковский, улица Ленина, 61А",  
 389.0: "Россия, Санкт-Петербург, улица Оптиков, 3А",  
 554.0: "Россия, Москва, Новомосковский административный округ, посёлок Внуково, 2-я Рейсовая улица, 2к5",  
 555.0: "Россия, Москва, Новомосковский административный округ, посёлок Внуково, 2-я Рейсовая улица, 2к5",  
 665.0: "Россия, Москва, Фестивальная улица, 13к1",  
 666.0: "Россия, Москва, Фестивальная улица, 13к1",
 668.0: "Россия, Москва, Фестивальная улица, 13к1",  
 669.0: "Россия, Москва, Фестивальная улица, 13к1",  
 952.0: "Россия, Москва, Свободный проспект, 20А",  
 953.0: "Россия, Москва, Свободный проспект, 20А",  
 954.0: "Россия, Москва, Свободный проспект, 20А",  
 955.0: "Россия, Москва, Свободный проспект, 20А",  
 956.0: "Россия, Москва, Свободный проспект, 20А", 
 1086.0: "Россия, Москва, Ленинградское шоссе, 16А", 
 1108.0: "Россия, Москва, Ленинградское шоссе, 16А",
 1226.0: "Россия, Москва, Свободный проспект, 20А",
 1761.0: "Россия, Омск, Омская улица, 33", 
 1792.0: "Россия, Ростов-на-Дону, Днепропетровская улица, 8", 
 1796.0: "Звёздная улица, 1, Санкт-Петербург",  
 841.0: "Россия, Иркутская область, Ангарск, 258-й квартал, 212", 
 1027.0: "Россия, Московская область, Балашиха, микрорайон Железнодорожный, Пролетарская улица, 2к2", 
 1246.0: "Россия, Москва, улица Земляной Вал, 3/1с7", 
 1302.0: "Россия, Самара, посёлок Соцгород, Долотный переулок, 16", 
 1368.0: "Россия, Московская область, Балашиха, микрорайон Железнодорожный, Пролетарская улица, 2к2", 
 1403.0: "Россия, Санкт-Петербург, улица Кораблестроителей, 21",
 1410.0: "Россия, Московская область, Реутов, Южная улица, 10А", 
 1562.0: "улица Ленина, 7, территория Цемдолина, Новороссийск, Краснодарский край", 
 1620.0: "Россия, Санкт-Петербург, Выборгское шоссе, 1", 
 2317.0: "Россия, Самара, Ново-Садовая улица, 363Б", 
 2470.0: "Россия, Липецкая область, Елец, Московское шоссе, 3А", 
 2516.0: "Россия, Москва, улица Коровий Вал, 9Б", 
 2642.0: "Россия, Томск, Комсомольский проспект, 13Б", 
 2833.0: "Россия, Новосибирск, улица Забалуева, 49/1", 
 8456.0: "Московская улица, 61, Истра, Московская область",
 4882.0: "Россия, Амурская область, Благовещенск, Амурская улица, 241", 
 4940.0: "Россия, Московская область, Подольск, микрорайон Климовск, улица Победы, 3", 
 5002.0: "Россия, Нижний Новгород, бульвар Мира, 14", 
 5017.0: "Россия, Томск, улица Смирнова, 15", 
 5035.0: "Россия, Москва, Новомосковский административный округ, Филимонковский район, Московский, 1-й микрорайон, 5Б", 
 5043.0: "Россия, Московская область, Клин, улица Гагарина, 8", 
 5133.0: "Россия, Краснодарский край, Сочи, микрорайон Центральный, Советская улица, 26А", 
 5153.0: "Россия, Краснодарский край, Сочи, жилой район Хоста, улица 50 лет СССР, 17", 
 5191.0: "Россия, Ульяновск, улица Урицкого, 100Д",
 5214.0: "Россия, Свердловская область, Екатеринбург, улица Восстания, 56к4", 
 5222.0: "Россия, Ленинградская область, Волхов, улица Пирогова, 12", 
 5247.0: "Солнечная улица, 19/1, посёлок Зональная Станция, Томский район", 
 5318.0: "Россия, Самара, Волжский проспект, 10", 
 5335.0: "Россия, Курск, улица Ленина, 60", 
 5336.0: "Россия, Курск, улица Дзержинского, 40", 
 5359.0: "Россия, Омск, улица Серова, 13", 
 5409.0: "Россия, Ставропольский край, городской округ Железноводск, посёлок Иноземцево, Советская улица, 95", 
 5415.0: "Россия, Амурская область, Благовещенск, улица Ломоносова, 160",
 5424.0: "Россия, Тульская область, Новомосковск, Комсомольская улица, 1", 
 5427.0: "Россия, Алтайский край, Барнаул, улица Малахова, 86В", 
 5458.0: "Россия, Пенза, Московская улица, 9", 
 5538.0: "Россия, Белгородская область, Яковлевский муниципальный округ, Строитель, улица 5 Августа, 28", 
 5601.0: "Россия, Амурская область, Благовещенск, улица Пушкина, 189", 
 5676.0: "Президентский бульвар, 20, Чебоксары, Чувашская Республика — Чувашия", 
 5801.0: "Россия, Алтайский край, Бийск, Коммунарский переулок, 23", 
 5826.0: "Россия, Республика Бурятия, Северобайкальск, проспект 60 лет СССР, 19", 
 5853.0: "Россия, Алтайский край, Барнаул, улица Попова, 70Д",
 5861.0: "Россия, Московская область, Пушкино, улица Чехова, 14А", 
 5864.0: "Россия, Московская область, Пушкино, улица Чехова, 14А", 
 6001.0: "Рабочая улица, 2А, посёлок городского типа Заиграево, городское поселение Заиграево, Заиграевский район, Республика Бурятия", 
 6030.0: "Россия, Тула, проспект Ленина, 113", 
 6055.0: "Россия, Тюменская область, Тобольск, 7-й микрорайон, 30", 
 6068.0: "Россия, Воронежская область, Лиски, улица Титова, 16А", 
 6157.0: "Россия, Алтайский край, Бийск, Коммунарский переулок, 23", 
 6158.0: "Россия, Алтайский край, Бийск, Коммунарский переулок, 23", 
 6159.0: "Россия, Алтайский край, Бийск, Коммунарский переулок, 23",
 6212.0: "Россия, Республика Татарстан (Татарстан), Казань, улица Декабристов, 185", 
 6213.0: "Россия, Республика Татарстан (Татарстан), Казань, улица Декабристов, 185", 
 6214.0: "Россия, Республика Татарстан (Татарстан), Казань, улица Декабристов, 185", 
 6517.0: "Россия, Московская область, городской округ Серпухов, Протвино, улица Ленина, 24Б", 
 6518.0: "Россия, Московская область, городской округ Серпухов, Протвино, улица Ленина, 24Б", 
 6519.0: "Россия, Московская область, городской округ Серпухов, Протвино, улица Ленина, 24Б", 
 6697.0: "Россия, Республика Башкортостан, Иглинский сельсовет, село Иглино, улица Ленина, 54", 
 6864.0: "Россия, Костромская область, Шарья, улица Чапаева, 7В", 
 6902.0: "Россия, Тула, улица Максима Горького, 7",
 6994.0: "Россия, Москва, Нагорный проезд, 5", 
 7057.0: "Россия, Киров, улица Ленина, 169", 
 7126.0: "Россия, Кемеровская область — Кузбасс, Ленинск-Кузнецкий, улица Ленина, 5А", 
 7147.0: "Россия, Липецкая область, Грязи, улица Крылова, 9", 
 7220.0: "Россия, Хабаровск, Матвеевское шоссе, 28", 
 7259.0: "Россия, Забайкальский край, Чита, улица Горбунова, 13", 
 7294.0: "Россия, Сахалинская область, Южно-Сахалинск, планировочный район Большая Елань, Авиационная улица, 2", 
 7332.0: "Россия, Красноярск, улица Калинина, 2В",
 7402.0: "Россия, Краснодарский край, Сочи, Виноградный переулок, 2", 
 7447.0: "Россия, Приморский край, Артём, улица Фрунзе, 48", 
 7487.0: "Россия, Нижний Новгород, Камчатский переулок, 7", 
 7566.0: "Россия, Курская область, Курчатовский район, рабочий посёлок имени Карла Либкнехта, улица Ленина, 33А", 
 7732.0: "улица Маршала Жукова, 5, городок Сосновый Бор, Улан-Удэ, Республика Бурятия", 
 7771.0: "Россия, Иркутск, бульвар Рябикова, 32Г", 
 7776.0: "Россия, Липецкая область, Елец, Московское шоссе, 3А", 
 7830.0: "Россия, Великий Новгород, улица Державина, 19", 
 7910.0: "Россия, Амурская область, Благовещенск, улица Шевченко, 36",
 7926.0: "Россия, Республика Башкортостан, Уфа, улица Менделеева, 137", 
 7974.0: "Россия, Московская область, Пушкино, улица Чехова, 14А", 
 8029.0: "Россия, Московская область, Пушкино, улица Чехова, 14А", 
 8083.0: "улица Карла Маркса, 38, Киров, Калужская область", 
 8130.0: "Россия, Магадан, проспект Ленина, 30Б", 
 8131.0: "Россия, Магадан, проспект Ленина, 30Б", 
 8132.0: "Россия, Магадан, проспект Ленина, 30Б", 
 8133.0: "Россия, Магадан, проспект Ленина, 30Б", 
 8307.0: "Россия, Тульская область, городской округ Тула, рабочий посёлок Плеханово, Заводская улица, 14",
 3002.0: "Россия, Приморский край, Хасанский муниципальный округ, посёлок городского типа Славянка, Ленинская улица, 64", 
 3154.0: "Россия, Московская область, городской округ Домодедово, аэропорт Домодедово имени М.В. Ломоносова, 22/1", 
 3319.0: "Россия, Хабаровский край, район им. Лазо, рабочий посёлок Мухен, улица Ленина, 17А", 
 3597.0: "Россия, Москва, Озерковская набережная, 28с2", 
 3604.0: "Россия, Москва, Шереметьевская улица, 60А", 
 3943.0: "Россия, Самарская область, Сызрань, улица Карла Маркса, 15", 
 4169.0: "Россия, Хабаровский край, Комсомольск-на-Амуре, Северное шоссе, 151", 
 4183.0: "Россия, Приморский край, Артём, Каширская улица, 23/1",
 2944.0: "Россия, Омская область, рабочий посёлок Нововаршавка, Красноармейская улица, 2В",
 4582.0: "Россия, Краснодар, микрорайон Центральный, улица Короленко, 2", 
 4660.0: "Россия, Камчатский край, Петропавловск-Камчатский, Кавказская улица, 49", 
 4670.0: "Россия, Камчатский край, Петропавловск-Камчатский, Космический проезд, 3А", 
 4693.0: "Россия, Красноярский край, Манско-Уярский округ, Уяр, улица 30 лет Победы, 21В", 
 8583.0: "Россия, Алтайский край, Троицкий район, село Троицкое, проспект Ленина, 19А", 
 8611.0: "Россия, Камчатский край, Петропавловск-Камчатский, Пограничная улица, 2/2", 
 8669.0: "Россия, Нижегородская область, рабочий посёлок Ардатов, улица Зуева, 38А", 
 8757.0: "Россия, Астраханская область, село Икряное, Зелёная улица, 2А",
 10.0: "проспект Ветеранов, 3, Краснокаменск, Забайкальский край",   
 14.0: "Россия, Брянская область, Жуковка, улица Карла Либкнехта, 15",  
 290.0: "Россия, Свердловская область, Екатеринбург, улица Красных Командиров, 19",  
 184.0: "Россия, Свердловская область, Нижний Тагил, проспект Ленина, 69",  
 564.0: "Киевское шоссе, 22-й километр, 6с1, район Солнцево, Западный административный округ, Москва",  
 647.0: "Россия, Москва, 1-я Тверская-Ямская улица, 32",  
 736.0: "Россия, Москва, улица Большая Ордынка, 21с2", 
 1021.0: "улица Земляной Вал, 14-16с1, Москва", 
 1625.0: "Россия, Нижний Новгород, Казанское шоссе, 12", 
 1890.0: "Россия, Нижний Новгород, улица Пискунова, 45", 
 1891.0: "Россия, Нижний Новгород, улица Пискунова, 45",  
 764.0: "улица 20-я Линия, 4, Нахичевань, Ростов-на-Дону",  
 785.0: "Россия, Иркутская область, Усолье-Сибирское, проспект Красных Партизан, 20", 
 1364.0: "Московское шоссе, 5А, Рязань", 
 1402.0: "Россия, Санкт-Петербург, Малая Балканская улица, 27", 
 1464.0: "Россия, Нижний Новгород, улица Родионова, 165", 
 1553.0: "Россия, Москва, Бирюлёвская улица, 51к1", 
 2154.0: "Россия, Новосибирск, площадь Карла Маркса, 7", 
 2287.0: "улица Энергетиков, 14, посёлок городского типа Ясногорск, Оловяннинский муниципальный округ, Забайкальский край", 
 2371.0: "Россия, Хабаровский край, Комсомольск-на-Амуре, Северное шоссе, 151", 
 2451.0: "Россия, Сахалинская область, Поронайский городской округ, село Восток, улица Гагарина, 9А", 
 2457.0: "Россия, Сахалинская область, Южно-Сахалинск, Детская улица, 4", 
 2526.0: "Новорижское шоссе, 26-й километр, городской округ Красногорск, Московская область", 
 2535.0: "Россия, Москва, Смоленская улица, 3", 
 2632.0: "Россия, Московская область, Раменский муниципальный округ, посёлок городского типа Быково, Советская улица, 1", 
 2651.0: "Новорижское шоссе, 22-й километр, вл1с1, Москва", 
 2768.0: "Россия, Красноярский край, городской округ ЗАТО Железногорск, посёлок Подгорный, улица Мира, 13", 
 2809.0: "Россия, Красноярск, Комсомольский проспект, 1Б", 
 8430.0: "Россия, Пермь, улица Ленина, 60", 
 4788.0: "Россия, Самарская область, Тольятти, Юбилейная улица, 2Г", 
 4821.0: "Россия, Московская область, Жуковский, улица Фрунзе, 1", 
 4887.0: "4-й микрорайон, 11, посёлок Текстильщиков, Чита, Забайкальский край", 
 4911.0: "Россия, Московская область, Одинцовский городской округ, деревня Лапино, 1-е Успенское шоссе", 
 4957.0: "Россия, Республика Башкортостан, Уфа, Бельская улица, 70", 
 4972.0: "Россия, Ростовская область, Батайск, улица Кирова, 51Б", 
 4988.0: "Россия, Приморский край, Артём, улица Гагарина, 133", 
 5196.0: "Россия, Киров, улица Сурикова, 3", 
 5223.0: "Южная улица, 5, Бугры, Всеволожский район, Ленинградская область", 
 5330.0: "Россия, Пенза, улица Кижеватова, 27Б", 
 5366.0: "Россия, Республика Башкортостан, Кумертау, улица Калинина, 2", 
 5386.0: "улица Чаадаева, 5Д, Нижний Новгород", 
 5454.0: "Россия, Псков, улица Труда, 37А", 
 5489.0: "Россия, Амурская область, Благовещенск, улица Островского, 14", 
 5507.0: "Россия, Камчатский край, Петропавловск-Камчатский, улица Ларина, 36", 
 5537.0: "Россия, Белгород, Садовая улица, 65В", 
 5553.0: "Россия, Республика Татарстан (Татарстан), Казань, улица Седова, 16", 
 5698.0: "Россия, Пермский край, Большесосновский муниципальный округ, село Петропавловск, улица Ленина", 
 5993.0: "Россия, Амурская область, Селемджинский район, посёлок городского типа Февральск, Енисейская улица, 7Б", 
 6113.0: "Россия, Приморский край, Владивосток, Русская улица, 39Б", 
 6245.0: "Россия, Тульская область, Алексин, улица Ленина, 14", 
 6246.0: "Россия, Тульская область, Алексин, улица Ленина, 14", 
 6258.0: "Россия, Нижний Новгород, улица Бекетова, 13А", 
 6259.0: "Россия, Нижний Новгород, улица Бекетова, 13А", 
 6260.0: "Россия, Нижний Новгород, улица Бекетова, 13А", 
 6261.0: "Россия, Нижний Новгород, улица Бекетова, 13А", 
 6266.0: "Россия, Приморский край, Владивосток, Русская улица, 39Б", 
 6381.0: "Россия, Нижний Новгород, проспект Ленина, 42", 
 6382.0: "Россия, Нижний Новгород, проспект Ленина, 42", 
 6383.0: "Россия, Нижний Новгород, проспект Ленина, 42", 
 6384.0: "Россия, Нижний Новгород, проспект Ленина, 42", 
 6647.0: "Россия, Нижний Новгород, Южное шоссе, 2", 
 6891.0: "Россия, Астраханская область, Приволжский район, село Началово, улица Ленина, 47", 
 7025.0: "Россия, Тула, улица Макаренко, 9", 
 7195.0: "улица Калинина, 1В, посёлок городского типа Дарасун, Карымский муниципальный округ, Забайкальский край", 
 7295.0: "улица Мира, 5А, посёлок Центральный, городской округ Барнаул, Алтайский край", 
 7367.0: "Россия, Республика Татарстан (Татарстан), Казань, Залесная улица, 66", 
 7394.0: "Россия, Пермь, улица Докучаева, 42Б", 
 7530.0: "Вокзальная улица, 17, посёлок городского типа Ерофей Павлович, Сковородинский муниципальный округ, Амурская область", 
 7547.0: "Новостроящаяся улица, 1, Шимановск, Амурская область", 
 7575.0: "Россия, Пенза, улица Терновского, 100А", 
 7595.0: "Россия, Волгоград, улица 64-й Армии, 4", 
 7663.0: "Россия, Амурская область, Тында, Спортивная улица, 1", 
 7680.0: "Россия, Волгоградская область, рабочий посёлок Средняя Ахтуба, Вятская улица, 17", 
 7683.0: "улица Карла Маркса, 26, посёлок городского типа Безенчук, Самарская область", 
 7721.0: "Россия, Иркутская область, Зима, Садовая улица, 26", 
 7756.0: "улица Ленина, 241, село Барышево, Новосибирский район", 
 7900.0: "улица Ленина, 12, Анжеро-Судженск, Кемеровская область — Кузбасс", 
 7901.0: "улица Ленина, 12, Анжеро-Судженск, Кемеровская область — Кузбасс", 
 8071.0: "Россия, Приморский край, Владивосток, Русская улица, 39Б", 
 8292.0: "Россия, Тула, улица Металлургов, 65А", 
 8314.0: "Россия, Красноярск, Ястынская улица, 19Д", 
 3105.0: "Россия, Красноярский край, Таймырский Долгано-Ненецкий муниципальный округ, Дудинка, улица Щорса, 29", 
 3634.0: "Россия, Санкт-Петербург, Пулковское шоссе, 35к6", 
 3795.0: "Советская улица, 14, посёлок городского типа Каменск, Кабанский район, Республика Бурятия", 
 4005.0: "Россия, Амурская область, Благовещенск, Нагорная улица, 19", 
 4014.0: "Россия, Кемерово, посёлок Комиссарово, Центральная улица, 159", 
 4098.0: "Россия, Московская область, Коломна, проспект Кирова, 15А", 
 4138.0: "Россия, Нижний Новгород, улица Веденяпина, 2Б", 
 4171.0: "Россия, Камчатский край, Петропавловск-Камчатский, проспект Циолковского, 42", 
 4206.0: "Россия, Амурская область, Благовещенск, улица Чайковского, 1/1", 
 2940.0: "улица Декабристов, 1, Казань, Республика Татарстан (Татарстан)",
 1991.0: "Пролетарская улица, 11, посёлок железнодорожной станции Высокая Гора, Красносельское сельское поселение, Высокогорский район, Республика Татарстан (Татарстан)",
 4554.0: "1-й микрорайон, 32, посёлок городского типа Камские Поляны, Нижнекамский район, Республика Татарстан (Татарстан)",
 343.0: "49-й комплекс, 22А, Набережные Челны, Республика Татарстан (Татарстан)",
 348.0: "Московский проспект, 128А, Набережные Челны, Республика Татарстан (Татарстан)",
 1958.0: "Набережночелнинский проспект, 88, Набережные Челны, Республика Татарстан (Татарстан)",
 4213.0: "проспект Ленина, 90/1, Кемерово", 
 4387.0: "Россия, Орловская область, Ливны, улица Энергетиков, 1А", 
 4403.0: "Россия, Республика Саха (Якутия), Якутск, улица Кирова, 18", 
 8531.0: "Россия, Сахалинская область, Южно-Сахалинск, Пограничная улица, 55", 
 2030.0: "Россия, Амурская область, Завитинск, улица Куйбышева, 27/72", 
 2904.0: "Красная улица, 15, станица Новокорсунская, Новокорсунское сельское поселение, Тимашевский район, Краснодарский край", 
 4500.0: "улица Поцелуева, 2, рабочий посёлок Мокшан, Пензенская область", 
 4592.0: "Красноармейская улица, 6, посёлок Красная Горбатка, Селивановский муниципальный округ, Владимирская область", 
 4597.0: "Россия, Республика Башкортостан, село Бижбуляк, Советская улица, 25",
 4647.0: "Россия, Камчатский край, Елизовский муниципальный округ, посёлок Раздольный, Советская улица, 4", 
 8522.0: "Россия, Республика Тыва, Бай-Тайгинский кожуун, село Тээли, улица Мугур, 12А", 
 8739.0: "Россия, Новосибирская область, рабочий посёлок Коченёво, Октябрьская улица, 8", 
 8764.0: "Россия, Республика Тыва, Улуг-Хемский кожуун, Шагонар, улица Дружбы, 59А/1",  
 115.0: "Россия, Республика Тыва, Дзун-Хемчикский кожуун, Чадан, улица Ленина, 31",  
 392.0: "Россия, Иркутская область, Усолье-Сибирское, улица Ленина, 83",  
 429.0: "Россия, Приморский край, Уссурийск, Колхозная улица, 10",  
 475.0: "12-й км тракт Томск-Итатка, с2, Томская область",  
 552.0: "Россия, Москва, Шоссейная улица, 34", 
 824.0: "Россия, Московская область, Люберцы, Смирновская улица, 16/179",  
 835.0: "Россия, Московская область, Люберцы, Смирновская улица, 16/179",  
 846.0: "Россия, Московская область, Люберцы, Смирновская улица, 16/179",  
 856.0: "Россия, Московская область, Люберцы, Смирновская улица, 16/179", 
 1692.0: "Россия, Ростов-на-Дону, микрорайон Северный, проспект Космонавтов, 15", 
 1693.0: "Россия, Ростов-на-Дону, микрорайон Северный, проспект Космонавтов, 15", 
 1838.0: "Россия, Волгоградская область, Волжский, улица Александрова, 18А",  
 721.0: "Россия, Москва, Зеленодольская улица, 31к1",  
 763.0: "улица 20-я Линия, 4, Нахичевань, Ростов-на-Дону", 
 867.0: "Россия, Волгоград, Невская улица, 11А",  
 868.0: "Россия, Красноярский край, Железногорск, улица Свердлова, 22",  
 891.0: "Россия, Красноярский край, Железногорск, улица Свердлова, 22", 
 1094.0: "Пятницкое шоссе, 1с1, посёлок городского типа Отрадное, городской округ Красногорск, Московская область", 
 1118.0: "Россия, Москва, Комсомольская площадь, 6соор1",
 1328.0: "Сибирский проспект, 7, Омск", 
 1342.0: "Россия, Нижний Новгород, Московское шоссе, 12", 
 1359.0: "Россия, Новосибирская область, Искитимский район, городское поселение Линёво, рабочий поселок Линёво, Коммунистический проспект, 2", 
 1413.0: "Россия, Москва, Новоясеневский проспект, 1",
 1442.0: "Россия, Москва, Россошанский проезд, 3", 
 1444.0: "Манежная площадь, 1с2, Москва", 
 1523.0: "Россия, Омский район, село Лузино, улица Майорова, 18", 
 1530.0: "Россия, Москва, Святоозёрская улица, 1А", 
 1593.0: "Россия, Санкт-Петербург, Комендантский проспект, 12В", 
 2049.0: "Россия, Тула, улица Максима Горького, 7А", 
 2119.0: "проспект Дзержинского, 33с1, Оренбург", 
 2142.0: "Россия, Свердловская область, Верхняя Пышма, Успенский проспект, 131В",
 2188.0: "Россия, Омск, улица 20-я Линия, 61к1", 
 2360.0: "проспект Ленина, 8, Нерюнгри, Республика Саха (Якутия)", 
 2368.0: "улица Калинина, 7, Комсомольск-на-Амуре, Хабаровский край", 
 2454.0: "Россия, Сахалинская область, Южно-Сахалинск, улица Чехова, 78", 
 2532.0: "Россия, Москва, улица Красного Маяка, 2Б", 
 2593.0: "Россия, Тюмень, Заречный проезд, 33", 
 2721.0: "Россия, Алтайский край, Бийск, Коммунарский переулок, 18", 
 8400.0: "Россия, Хабаровский край, Комсомольск-на-Амуре, проспект Ленина, 50", 
 4831.0: "Россия, Новосибирская область, Обь, улица Ломоносова, 20/1",
 4836.0: "Россия, Киров, Пролетарская улица, 22", 
 4854.0: "Россия, Свердловская область, Екатеринбург, улица Титова, 17В", 
 5294.0: "Россия, Костромская область, посёлок городского типа Красное-на-Волге, Советская улица, 52", 
 5422.0: "Россия, Пенза, проспект Строителей, 2Б", 
 5449.0: "Россия, Москва, Филёвский бульвар, 10", 
 5455.0: "Россия, Камчатский край, Петропавловск-Камчатский, улица Ларина, 7/1", 
 5550.0: "Россия, Забайкальский край, Чита, улица Недорезова, 1М", 
 5551.0: "Россия, Амурская область, Благовещенск, Игнатьевское шоссе, 21", 
 5594.0: "Россия, Тульская область, Алексин, улица Мира, 20",
 5603.0: "Россия, Камчатский край, Петропавловск-Камчатский, Пограничная улица, 13", 
 5710.0: "Россия, Томская область, Северск, улица Победы, 12Б", 
 5889.0: "Россия, Волгоградская область, Волжский, улица Мира, 36Б", 
 5924.0: "Россия, Сахалинская область, Южно-Сахалинск, улица Есенина, 15А", 
 6004.0: "Россия, Тюмень, улица Республики, 171", 
 6009.0: "Россия, Тюмень, улица Республики, 171", 
 6010.0: "Россия, Тюмень, улица Республики, 171", 
 6011.0: "Россия, Тюмень, улица Республики, 171", 
 6032.0: "Россия, Тульская область, Щёкино, Советская улица, 2Б",
 6039.0: "Россия, Тульская область, Венёв, микрорайон Южный, 68", 
 6250.0: "Россия, Республика Башкортостан, Уфа, улица Правды, 21", 
 6251.0: "Россия, Республика Башкортостан, Уфа, улица Правды, 21", 
 6252.0: "Россия, Республика Башкортостан, Уфа, улица Правды, 21", 
 6315.0: "Россия, Камчатский край, Петропавловск-Камчатский, проспект Рыбаков, 2/1", 
 6357.0: "Россия, Хабаровск, Амурский бульвар, 2", 
 6547.0: "Россия, Киров, микрорайон Коминтерновский, улица Баумана, 5", 
 6682.0: "Россия, Республика Карелия, Петрозаводск, улица Правды, 38Б", 
 6683.0: "Россия, Кемеровская область — Кузбасс, Новокузнецк, улица Пржевальского, 2",
 6692.0: "Россия, Московская область, Клин, улица Мира, 58/25", 
 6942.0: "Россия, Оренбургская область, Кувандык, Вокзальная улица, 18", 
 7138.0: "Никольская улица, 1А, рабочий посёлок Майский, Советско-Гаванский район, Хабаровский край", 
 7228.0: "улица Ватутина, 7В, Омск", 
 7244.0: "Южная улица, 20, село Екатеринославка, Октябрьский муниципальный округ, Амурская область", 
 7353.0: "Россия, Сахалинская область, Южно-Сахалинск, Сахалинская улица, 54", 
 7397.0: "Россия, Краснодарский край, Сочи, улица Ленина, 173", 
 7470.0: "Россия, Красноярский край, Ачинск, 4-й микрорайон, 8", 
 7491.0: "Россия, Ставропольский край, Андроповский муниципальный округ, село Курсавка, Красная улица, 13",
 7505.0: "Россия, Краснодарский край, Тихорецк, Шоссейная улица, 3", 
 7555.0: "улица Бараниха, 10, посёлок городского типа Нижняя Пойма, Иланско-Нижнеингашский округ, Красноярский край", 
 7593.0: "Россия, Самарская область, Кинель, Советская улица, 64", 
 7632.0: "Россия, Хабаровск, Амурский бульвар, 39", 
 7684.0: "Россия, Ханты-Мансийский автономный округ — Югра, Сургут, Югорский тракт, 2", 
 7686.0: "1-й Станционный проезд, 7Ас9, Саратов", 
 8149.0: "Россия, Курск, улица Карла Маркса, 59", 
 8156.0: "Россия, Тула, улица Шухова, 26", 
 8165.0: "Россия, Воронеж, улица Туполева, 5У",
 8330.0: "Россия, Камчатский край, Петропавловск-Камчатский, проспект Рыбаков, 2/1",
 42.0: "Россия, Москва, Волоколамское шоссе, 2",   
 46.0: "Россия, Великий Новгород, район Торговая сторона, улица Фёдоровский Ручей, 2Г",   
 91.0: "Россия, Краснодарский край, Абинский район, Фёдоровское сельское поселение, станица Фёдоровская, Первомайская улица, 4",  
 129.0: "Россия, Амурская область, Белогорск, улица Кирова, 188",  
 149.0: "Октябрьская улица, 38Б, село Тербуны, Липецкая область",  
 187.0: "Комсомольский проспект, 2А, Челябинск",  
 269.0: "Комсомольский проспект, 2А, Челябинск",  
 270.0: "Комсомольский проспект, 2А, Челябинск",  
 271.0: "проспект Ленина, 22, Нижний Тагил, Свердловская область",
 282.0: "Россия, Свердловская область, Екатеринбург, площадь Бахчиванджи, 1",  
 302.0: "Россия, Санкт-Петербург, Большая Морская улица, 14",  
 310.0: "Россия, Москва, улица Покровка, 31с1",  
 312.0: "Россия, Москва, Варшавское шоссе, 1с1-2",  
 359.0: "Россия, Камчатский край, Петропавловск-Камчатский, Космический проезд, 3А",  
 400.0: "Россия, Санкт-Петербург, проспект Просвещения, 19",  
 619.0: "Россия, Москва, Мясницкая улица, 13с1",  
 620.0: "Россия, Москва, Мясницкая улица, 13с1",  
 621.0: "Россия, Москва, Мясницкая улица, 13с1", 
 658.0: "Россия, Москва, Митинская улица, 36к1",  
 659.0: "Россия, Москва, Митинская улица, 36к1",  
 660.0: "Россия, Москва, Митинская улица, 36к1",  
 661.0: "Россия, Москва, Митинская улица, 36к1",  
 672.0: "Россия, Московская область, Мытищи, Шараповский проезд, 3А",  
 673.0: "Россия, Московская область, Мытищи, Шараповский проезд, 3А",  
 674.0: "Россия, Московская область, Мытищи, Шараповский проезд, 3А",  
 675.0: "Россия, Московская область, Мытищи, Шараповский проезд, 3А",  
 676.0: "Россия, Московская область, Мытищи, Шараповский проезд, 3А", 
 693.0: "Автозаводская улица, 13/1, Москва",  
 695.0: "Автозаводская улица, 13/1, Москва",  
 696.0: "Россия, Москва, Варшавское шоссе, 76к1",  
 697.0: "Россия, Москва, Большая Тульская улица, 2",  
 698.0: "Россия, Москва, Большая Тульская улица, 2",  
 699.0: "Россия, Москва, Большая Тульская улица, 2",  
 701.0: "Россия, Москва, Азовская улица, 24к2",  
 702.0: "Россия, Москва, Азовская улица, 24к2",  
 703.0: "Россия, Москва, Азовская улица, 24к2", 
 712.0: "Россия, Москва, Варшавское шоссе, 160",  
 713.0: "Россия, Москва, Варшавское шоссе, 160",  
 714.0: "Россия, Москва, Варшавское шоссе, 160",  
 715.0: "Россия, Москва, бульвар Дмитрия Донского, 11",  
 716.0: "Россия, Москва, бульвар Дмитрия Донского, 11",  
 729.0: "Россия, Москва, улица Большая Ордынка, 21с2",  
 730.0: "Россия, Москва, улица Большая Ордынка, 21с2",  
 731.0: "Россия, Москва, улица Большая Ордынка, 21с2",  
 745.0: "Россия, Ростов-на-Дону, Темерницкая улица, 93", 
 748.0: "Россия, Ростов-на-Дону, Темерницкая улица, 93",  
 772.0: "Россия, Самара, улица Стара-Загора, 124А",  
 773.0: "Россия, Самара, улица Стара-Загора, 124А",  
 808.0: "Россия, Омск, улица Богдана Хмельницкого, 165",  
 935.0: "Россия, Московская область, Балашиха, микрорайон Салтыковка, Носовихинское шоссе, вл4",  
 941.0: "Россия, Москва, 3-й Крутицкий переулок, 11",  
 942.0: "Россия, Москва, 3-й Крутицкий переулок, 11",  
 943.0: "Россия, Москва, 3-й Крутицкий переулок, 11",  
 978.0: "Киевское шоссе, 22-й километр, 4с1кА, район Солнцево, Западный административный округ, Москва", 
 988.0: "Россия, Москва, Варшавское шоссе, 76к1", 
 1016.0: "улица Земляной Вал, 14-16с1, Москва", 
 1031.0: "улица Земляной Вал, 14-16с1, Москва", 
 1040.0: "Россия, Республика Татарстан (Татарстан), Набережные Челны, проспект Мусы Джалиля, 58", 
 1043.0: "Россия, Оренбургская область, Орск, Орский проспект, 7", 
 1044.0: "Россия, Ростов-на-Дону, микрорайон Заречная, Пойменная улица, 1", 
 1048.0: "Россия, Нижний Новгород, проспект Ленина, 33", 
 1059.0: "Россия, Москва, улица Маршала Бирюзова, 32", 
 1143.0: "Россия, Нижний Новгород, улица Героя Юрия Смирнова, 14", 
 1157.0: "Россия, Нижний Новгород, жилой район Мещерское Озеро, улица Сергея Есенина, 30", 
 1210.0: "Россия, Санкт-Петербург, площадь Стачек, 7", 
 1213.0: "192-й квартал, 12, Ангарск, Иркутская область", 
 1239.0: "Россия, Ханты-Мансийский автономный округ — Югра, Сургут, Югорский тракт, 38", 
 1290.0: "проспект Космонавтов, 6В, Барнаул, Алтайский край", 
 1310.0: "Россия, Чувашская Республика — Чувашия, Чебоксары, проспект Максима Горького, 10к1", 
 1344.0: "Россия, Москва, Славянский бульвар, 13к1", 
 1360.0: "Россия, Республика Татарстан (Татарстан), Набережные Челны, проспект Мира, 88/20", 
 1366.0: "Россия, Москва, Свободный проспект, 33", 
 1370.0: "Россия, Москва, улица Шолохова, 5к2", 
 1404.0: "Россия, Санкт-Петербург, шоссе Революции, 41/39", 
 1408.0: "Россия, Санкт-Петербург, Хасанская улица, 17к1", 
 1409.0: "проспект Маршала Жукова, 31к1, Санкт-Петербург", 
 1414.0: "Россия, Ханты-Мансийский автономный округ — Югра, Сургут, Югорский тракт, 2", 
 1439.0: "Варшавское шоссе, 87Б, Москва", 
 1454.0: "Россия, Санкт-Петербург, Фурштатская улица, 40", 
 1463.0: "Россия, Нижний Новгород, Казанское шоссе, 4А", 
 1466.0: "Россия, Нижний Новгород, улица Ванеева, 96Б", 
 1491.0: "Россия, Самарская область, Тольятти, улица Льва Яшина, 14", 
 1495.0: "Главная улица, 60, деревня Новое Девяткино, Всеволожский район, Ленинградская область", 
 1510.0: "Россия, Красноярский край, Железногорск, улица Решетнёва, 2", 
 1529.0: "Россия, Москва, Святоозёрская улица, 1А", 
 1592.0: "Россия, Нижний Новгород, улица Плотникова, 3А", 
 1655.0: "Россия, Санкт-Петербург, Выборгское шоссе, 19к1", 
 1697.0: "Россия, Республика Башкортостан, Уфа, улица Маршала Жукова, 29", 
 1698.0: "Россия, Республика Башкортостан, Уфа, улица Маршала Жукова, 29", 
 1758.0: "улица 12 Декабря, 115, Омск", 
 1779.0: "Россия, Воронеж, улица Владимира Невского, 17", 
 1785.0: "Россия, Нижний Новгород, улица Белинского, 61", 
 1786.0: "Россия, Нижний Новгород, улица Белинского, 61", 
 1794.0: "Россия, Краснодар, улица 1 Мая, 158", 
 1799.0: "Россия, Ростов-на-Дону, Западный, Коммунистический проспект, 30", 
 1813.0: "Россия, Санкт-Петербург, Большая Зеленина улица, 8",  
 1900.0: "Россия, Санкт-Петербург, Бухарестская улица, 89", 
 1909.0: "Россия, Нижегородская область, Балахна, Советская площадь, 14", 
 1984.0: "Россия, Великий Новгород, улица Зелинского, 52", 
 2005.0: "Россия, Краснодарский край, Брюховецкий район, станица Брюховецкая, улица Ленина, 36", 
 2008.0: "Россия, Самарская область, сельское поселение Кинель-Черкассы, село Кинель-Черкассы, Куйбышевская улица, 54А", 
 2009.0: "Россия, Красноярский край, Манско-Уярский округ, село Шалинское, улица Ленина, 28", 
 2026.0: "Россия, Забайкальский край, Чита, Александро-Заводская улица, 21", 
 2084.0: "Россия, Томский район, посёлок Зональная Станция, улица 40 лет Победы, 1Б", 
 2098.0: "Россия, Оренбург, проспект Победы, 140В", 
 2100.0: "Россия, Оренбург, Раздольная улица, 3", 
 2102.0: "Россия, Оренбургская область, Новотроицк, Советская улица, 93А", 
 2128.0: "Россия, Ставропольский край, Кисловодск, проспект Карла Маркса, 1",
 2169.0: "Россия, Омск, 24-я Северная улица, 125А", 
 2246.0: "Россия, Красноярский край, Железногорск, улица Королёва, 6А", 
 2261.0: "Россия, Красноярский край, Ачинск, улица Кравченко, 7А", 
 2262.0: "улица Ленинского Комсомола, 26А, Сосновоборск, Красноярский край", 
 2282.0: "Россия, Республика Бурятия, Улан-Удэ, улица Яковлева, 8", 
 2293.0: "проспект Кирова, 52, Ленинск-Кузнецкий, Кемеровская область — Кузбасс", 
 2295.0: "Россия, Кемеровская область — Кузбасс, Ленинск-Кузнецкий, улица Пушкина, 90", 
 2301.0: "проспект Победы, 29, Новокуйбышевск, Самарская область", 
 2365.0: "улица Кирова, 28, Комсомольск-на-Амуре, Хабаровский край",
 2366.0: "Россия, Хабаровский край, Комсомольск-на-Амуре, проспект Первостроителей, 21", 
 2373.0: "Россия, Амурская область, Благовещенск, улица Горького, 182", 
 2383.0: "улица Карла Маркса, 20, Свободный, Амурская область", 
 2392.0: "Россия, Приморский край, Хасанский муниципальный округ, посёлок городского типа Посьет, Портовая улица, 41", 
 2394.0: "Россия, Камчатский край, Петропавловск-Камчатский, Космический проезд, 3А", 
 2442.0: "Россия, Сахалинская область, Южно-Сахалинск, Курильская улица, 38", 
 2443.0: "Россия, Сахалинская область, Южно-Сахалинск, проспект Победы, 30", 
 2444.0: "Россия, Сахалинская область, Холмск, Лесозаводская улица, 159", 
 2504.0: "Россия, Москва, Аргуновская улица, 5", 
 2517.0: "Россия, Тула, Менделеевская улица, 11", 
 2598.0: "Россия, Тюмень, улица Республики, 252", 
 2661.0: "Россия, Белгородская область, Старый Оскол, микрорайон Лесной, 1", 
 2671.0: "Русская улица, 2К, Владивосток, Приморский край", 
 2672.0: "Россия, Приморский край, Владивосток, Трамвайная улица, 14Б", 
 2684.0: "улица Мира, 205, посёлок городского типа Ильский, Ильское городское поселение, Северский район, Краснодарский край", 
 2753.0: "Россия, Омск, 6-я Станционная улица, 2/5",
 2790.0: "Центральная улица, 6, городской посёлок Берёзовка, городской округ Красноярск", 
 2846.0: "улица Димитрова, 14, Комсомольск-на-Амуре, Хабаровский край", 
 2852.0: "Россия, Кемеровская область — Кузбасс, Анжеро-Судженск, улица Желябова, 17", 
 2908.0: "Россия, Краснодарский край, Курганинск, Комсомольская улица, 28/8", 
 2922.0: "Советская улица, 34, рабочий посёлок Таврическое, Омская область", 
 2930.0: "Россия, Омская область, Русско-Полянский район, село Солнечное, улица Титова, 4", 
 2948.0: "Россия, Сахалинская область, Южно-Сахалинск, проспект Мира, 231", 
 2957.0: "Россия, Алтайский край, село Ключи, улица Кирова, 4", 
 2965.0: "Кооперативная улица, 48, станица Новомышастовская, Новомышастовское сельское поселение, Красноармейский район",
 3123.0: "Россия, Москва, Большой Кисловский переулок, 6", 
 3145.0: "Россия, Приморский край, Артём, улица Ленина, 6", 
 3155.0: "Россия, Республика Тыва, Кызыл, улица Дружбы, 13А", 
 3174.0: "Россия, Москва, Новомосковский административный округ, район Внуково, деревня Шарапово, Придорожная улица, 14с2", 
 3177.0: "Россия, Красноярский край, Таймырский Долгано-Ненецкий муниципальный округ, Дудинка, улица Островского, 9", 
 3191.0: "Россия, Камчатский край, Петропавловск-Камчатский, Пограничная улица, 2/2", 
 3212.0: "Россия, Астрахань, Аэропортовский проезд, 1с2", 
 3219.0: "Ново-Садовая улица, 160Д, Самара", 
 3224.0: "Россия, Московская область, Богородский городской округ, посёлок городского типа имени Воровского, Рабочая улица, 3Ас1",
 3309.0: "Россия, Еврейская автономная область, Биробиджанский район, село Бирофельд, Центральная улица, 52", 
 3350.0: "Россия, Брянск, проспект Станке Димитрова, 75", 
 3357.0: "Россия, Белгородская область, Губкин, улица Артёма, 12", 
 3457.0: "Россия, Ставропольский край, Пятигорск, Теплосерная улица, 21А", 
 3548.0: "Россия, Республика Хакасия, село Таштып, Советская улица, 157", 
 3561.0: "проспект имени Газеты Красноярский Рабочий, 59, Красноярск", 
 3562.0: "Чертановская улица, 1Вк1, Москва", 
 3573.0: "Чертановская улица, 1Вк1, Москва",
 3638.0: "Балканская площадь, 5Ю, Санкт-Петербург", 
 3655.0: "Россия, Москва, Новомосковский административный округ, район Коммунарка, деревня Саларьево, вл7с1", 
 3657.0: "Россия, Московская область, Мытищи, Шараповский проезд, вл2", 
 3661.0: "Россия, Москва, 3-я Рыбинская улица, 22с6", 
 3691.0: "Россия, Санкт-Петербург, Комендантский проспект, 11", 
 3778.0: "территория Складского комплекса ССТ, к11, Раменский муниципальный округ, Московская область", 
 3845.0: "Россия, Хабаровский край, рабочий посёлок Солнечный, улица Геологов, 24", 
 3914.0: "Россия, Тула, улица Фрунзе, 7А", 
 3980.0: "Россия, Республика Тыва, Кызыл, улица Дружбы, 21Б",
 4008.0: "Россия, Астрахань, проспект Губернатора Анатолия Гужвина, 10", 
 4009.0: "Россия, Республика Бурятия, Улан-Удэ, улица Шумяцкого, 3А", 
 4017.0: "Россия, Республика Хакасия, рабочий посёлок Усть-Абакан, улица Карла Маркса, 11А", 
 4050.0: "Россия, Хабаровск, Тихоокеанская улица, 134", 
 4115.0: "Ленинградская улица, 38, посёлок городского типа Прогресс, Амурская область", 
 4189.0: "Россия, Красноярский край, Лесосибирск, 5-й микрорайон, 1Б", 
 4223.0: "Россия, Амурская область, Тында, улица Красная Пресня, 52А", 
 4233.0: "Ленинская улица, 11, село Подгорное, Чаинский район, Томская область", 
 4245.0: "Россия, Республика Бурятия, Заиграевский район, сельское поселение Новоильинское, село Новоильинск, улица Ленина, 10",
 4280.0: "Россия, Иркутская область, Ангарск, 86-й квартал, 40", 
 4299.0: "Россия, Приморский край, Большой Камень, улица Лебедева, 1", 
 4300.0: "Россия, Приморский край, Большой Камень, улица Лебедева, 1", 
 4339.0: "Россия, Приморский край, Хасанский муниципальный округ, посёлок городского типа Зарубино, Молодёжная улица, 3", 
 4366.0: "Первомайский проспект, 53/2, Рязань", 
 4401.0: "Россия, Брянская область, Дятьково, улица Ленина, 182к4", 
 4410.0: "улица Кирова, 63, село Заброды, Калачеевский район, Воронежская область", 
 4430.0: "Центральная улица, 19, городской посёлок Берёзовка, городской округ Красноярск", 
 4448.0: "Россия, Забайкальский край, Сретенский район, посёлок городского типа Кокуй, Заводская улица, 9А",
 4456.0: "Россия, Воронежская область, село Верхний Мамон, улица 22-го Партсъезда, 46", 
 4462.0: "Россия, Сахалинская область, Южно-Сахалинск, улица Ленина, 213", 
 4483.0: "Россия, Пензенская область, рабочий посёлок Лунино, Кооперативная улица, 42А", 
 4515.0: "Россия, Республика Дагестан, Хасавюрт, Махачкалинское шоссе, 3А", 
 4528.0: "улица Луначарского, 19Б, Валдай, Новгородская область", 
 4531.0: "Россия, Курск, улица Щепкина, 4Б", 
 4538.0: "улица Карла Маркса, 11/23, Ульяновск", 
 4545.0: "Россия, Ульяновская область, рабочий посёлок Старая Майна, Революционная улица, 81", 
 4546.0: "Россия, Красноярский край, Ермаковский сельсовет, село Ермаковское, улица Карла Маркса, 36А",
 4556.0: "Россия, Красноярский край, Ачинск, 7-й микрорайон, 4Б",
 4604.0: "Россия, Сахалинская область, Холмск, площадь Ленина, 9", 
 4607.0: "Россия, Республика Адыгея (Адыгея), Тахтамукайский район, Старобжегокайское сельское поселение, аул Новая Адыгея, улица Тургеневское шоссе, 27", 
 4610.0: "Россия, Республика Адыгея (Адыгея), Тахтамукайский район, Старобжегокайское сельское поселение, аул Новая Адыгея, улица Тургеневское шоссе, 27", 
 4618.0: "Россия, Красноярск, улица Весны, 7Б",
 4644.0: "Россия, Ярославская область, Переславль-Залесский, Ростовская улица, 3", 
 4669.0: "Россия, Республика Татарстан (Татарстан), посёлок городского типа Рыбная Слобода, улица Ленина, 63А", 
 4686.0: "Россия, Рязанская область, рабочий посёлок Ухолово, улица Ленина, 24", 
 4729.0: "Россия, Краснодарский край, Сочи, улица Гастелло, 42", 
 4732.0: "Россия, Алтайский край, Барнаул, улица Гущина, 154Д",
 4769.0: "Россия, Приморский край, Артём, улица Урбанского, 6", 
 4800.0: "Россия, Забайкальский край, Чита, улица Лермонтова, 17", 
 4817.0: "Россия, Красноярск, улица Никитина, 1В", 
 4819.0: "Россия, Республика Башкортостан, Иглинский сельсовет, село Иглино, улица Горького, 41А", 
 4822.0: "улица Ленина, 17, хутор Трудобеликовский, Трудобеликовское сельское поселение, Красноармейский", 
 4829.0: "Россия, Приморский край, Октябрьский муниципальный округ, посёлок городского типа Липовцы, Угольная улица, 18", 
 4856.0: "Россия, Алтайский край, Барнаул, улица Юрина, 202В", 
 4862.0: "Россия, Пенза, Красная улица, 53", 
 4881.0: "Россия, Тульская область, Новомосковск, Рудничная улица, 16Б",
 4891.0: "Россия, Иркутский муниципальный округ, село Хомутово, Колхозная улица, 135", 
 4942.0: "Россия, Липецкая область, Грязинский район, Казинский сельсовет, село Казинка, Октябрьская улица, 23", 
 4955.0: "Россия, Нижний Новгород, улица Ванеева, 231", 
 5004.0: "Россия, Республика Саха (Якутия), Мирный, улица Тихонова, 14А", 
 5013.0: "Россия, Псков, Юбилейная улица, 43В", 
 5022.0: "Россия, Амурская область, Благовещенск, улица Горького, 237", 
 5028.0: "Россия, Сахалинская область, Южно-Сахалинск, улица Ленина, 173", 
 5046.0: "Россия, Москва, МКАД, 91-й километр, 1", 
 5081.0: "Россия, Нижний Новгород, улица Лескова, 40",
 5082.0: "Россия, Нижний Новгород, улица Ватутина, 3", 
 5084.0: "Россия, Нижний Новгород, улица Бетанкура, 1", 
 5135.0: "Россия, Великий Новгород, Московская улица, 7", 
 5136.0: "Россия, Нижний Новгород, Московское шоссе, 12", 
 5142.0: "Россия, Тамбов, Студенецкая улица, 20А", 
 5150.0: "Россия, Пенза, Звёздная улица, 1", 
 5151.0: "Россия, Краснодарский край, Сочи, микрорайон Центральный, Московская улица, 22", 
 5152.0: "Россия, Краснодарский край, городской округ Сочи, посёлок городского типа Красная Поляна, улица Турчинского, 24", 
 5178.0: "Россия, Тула, Майская улица, 1",
 5179.0: "Россия, Тула, Советская улица, 59", 
 5219.0: "Советская улица, 2В, Тосно, Ленинградская область", 
 5290.0: "Россия, Владимирская область, Муром, Орловская улица, 26В", 
 5292.0: "Россия, Костромская область, посёлок городского типа Ветлужский, улица Горького, 2", 
 5293.0: "Россия, Костромская область, посёлок городского типа Красное-на-Волге, Советская улица, 49", 
 5299.0: "Россия, Краснодарский край, городской округ Геленджик, село Кабардинка, улица Мира, 20", 
 5313.0: "Россия, Ставропольский край, Георгиевск, улица Калинина, 101/1", 
 5340.0: "Россия, Приморский край, Артём, улица Фрунзе, 21", 
 5358.0: "Россия, Камчатский край, Елизово, улица Завойко, 108",
 5360.0: "Россия, Омск, проспект Мира, 100", 
 5375.0: "Россия, Смоленская область, Вяземский муниципальный округ, село Вязьма-Брянская, Авиационная улица, 7", 
 5400.0: "Россия, Оренбургская область, Бузулук, 4-й микрорайон, 1Ж", 
 5423.0: "Россия, Тула, улица 9 Мая, 3", 
 5434.0: "Россия, Амурская область, Благовещенск, Советский переулок, 65/1", 
 5441.0: "Россия, Нижний Новгород, жилой район Северный, Львовская улица, 8", 
 5444.0: "Россия, Нижний Новгород, Коломенская улица, 6", 
 5457.0: "Россия, Курск, улица Карла Маркса, 68", 
 5463.0: "Россия, Республика Башкортостан, Октябрьский, улица Кортунова, 13А",
 5487.0: "Россия, Амурская область, Благовещенск, улица Ленина, 135", 
 5496.0: "Россия, Волгоградская область, Волжский, проспект имени Ленина, 313", 
 5506.0: "Россия, Приморский край, Надеждинский район, село Вольно-Надеждинское, улица Пушкина, 49", 
 5547.0: "Россия, Тамбов, Бастионная улица, 10", 
 5575.0: "Россия, Тула, проспект Ленина, 122", 
 5588.0: "Россия, Краснодарский край, Анапа, улица Тургенева, 244", 
 5599.0: "Россия, Амурская область, Благовещенск, улица Ленина, 190", 
 5615.0: "Россия, Рязань, улица Новосёлов, 21В", 
 5634.0: "Дорожная улица, 6А, посёлок Нагорный, Ярославский муниципальный округ",
 5661.0: "Россия, Пенза, Московская улица, 40", 
 5662.0: "Россия, Пенза, Московская улица, 40", 
 5663.0: "Россия, Пенза, Московская улица, 40", 
 5697.0: "Россия, Великий Новгород, Псковская улица, 33", 
 5749.0: "Россия, Республика Татарстан (Татарстан), Казань, улица Хусаина Мавлютова, 17", 
 5760.0: "Россия, Алтайский край, Барнаул, улица Малахова, 88В", 
 5761.0: "Россия, Алтайский край, Барнаул, улица Малахова, 88В", 
 5762.0: "Россия, Алтайский край, Барнаул, улица Малахова, 88В", 
 5763.0: "Россия, Алтайский край, Барнаул, улица Малахова, 88В",
 5765.0: "Россия, Великий Новгород, Великая улица, 22А", 
 5809.0: "Россия, Пензенская область, Заречный, проспект Мира, 5", 
 5813.0: "Россия, Ярославль, проспект Фрунзе, 56Д", 
 5820.0: "Россия, Кемеровская область — Кузбасс, Тайга, проспект Кирова, 28", 
 5847.0: "Россия, Амурская область, Благовещенск, Игнатьевское шоссе, 15", 
 5848.0: "Россия, Приморский край, Находка, Школьная улица, 1Б", 
 5909.0: "Россия, Белгородская область, Валуйки, Новая улица, 1", 
 5964.0: "Вокзальная улица, посёлок Горхон, сельское поселение Горхонское, Заиграевский район, Республика Бурятия", 
 5986.0: "Россия, Тула, Пушкинская улица, 32",
 5987.0: "Россия, Республика Саха (Якутия), Нерюнгринский район, посёлок городского типа Беркакит, 8", 
 5994.0: "Россия, Амурская область, Тындинский муниципальный округ, посёлок Юктали, Привокзальная улица, 3", 
 5995.0: "Россия, Камчатский край, Вилючинск, жилой район Рыбачий, улица Вилкова, 37", 
 5996.0: "Россия, Камчатский край, Вилючинск, жилой район Рыбачий, улица Вилкова, 37", 
 6031.0: "Россия, Тула, улица 9 Мая, 2", 
 6047.0: "Россия, Республика Татарстан (Татарстан), Казань, Советский район, посёлок Дербышки, улица Мира, 33", 
 6048.0: "Россия, Республика Татарстан (Татарстан), Казань, Советский район, посёлок Дербышки, улица Мира, 33", 
 6053.0: "Россия, Кировская область, Слободской, улица Грина, 10", 
 6077.0: "Россия, Сахалинская область, Южно-Сахалинск, улица Ленина, 234",
 6078.0: "Россия, Сахалинская область, Южно-Сахалинск, улица Ленина, 234", 
 6080.0: "Россия, Киров, улица Монтажников, 38", 
 6081.0: "Россия, Киров, Пролетарская улица, 15", 
 6144.0: "Россия, Свердловская область, Екатеринбург, улица Учителей, 2Б", 
 6160.0: "проспект Мира, 27В, Томск", 
 6193.0: "Россия, Амурская область, Благовещенск, Институтская улица, 1",
 6237.0: "Россия, Удмуртская Республика, Ижевск, Клубная улица, 76/8", 
 6272.0: "Россия, Нижний Новгород, Советская улица, 12", 
 6300.0: "Россия, Московская область, Павловский Посад, улица Кирова, 4", 
 6301.0: "Россия, Московская область, Павловский Посад, улица Кирова, 4", 
 6322.0: "Россия, Смоленск, улица Рыленкова, 35Б", 
 6323.0: "Россия, Смоленск, улица Рыленкова, 35Б", 
 6324.0: "Россия, Смоленск, улица Рыленкова, 35Б", 
 6353.0: "4-й микрорайон, 19, Тобольск, Тюменская область", 
 6354.0: "4-й микрорайон, 19, Тобольск, Тюменская область", 
 6355.0: "4-й микрорайон, 19, Тобольск, Тюменская область",
 6356.0: "4-й микрорайон, 19, Тобольск, Тюменская область", 
 6371.0: "Россия, Хабаровский край, Верхнебуреинский район, рабочий посёлок Новый Ургал, Киевская улица, 2А", 
 6415.0: "Россия, Республика Башкортостан, Уфа, улица Карла Маркса, 23", 
 6453.0: "Россия, Нижний Новгород, проспект Ленина, 111", 
 6454.0: "Россия, Нижний Новгород, проспект Ленина, 111", 
 6455.0: "Россия, Нижний Новгород, проспект Ленина, 111", 
 6456.0: "Россия, Нижний Новгород, проспект Ленина, 111", 
 6480.0: "площадь Менделеева, 2к135, Переславль-Залесский, Ярославская область, Россия, 152025", 
 6522.0: "Россия, Москва, Нахимовский проспект, 11к1",
 6523.0: "Россия, Москва, Нахимовский проспект, 11к1", 
 6589.0: "микрорайон Г, 1А, Пущино, городской округ Серпухов, Московская область", 
 6605.0: "Россия, Нижний Новгород, улица Минина, 19/6", 
 6606.0: "Россия, Нижний Новгород, улица Минина, 19/6", 
 6607.0: "Россия, Нижний Новгород, улица Минина, 19/6", 
 6617.0: "Россия, Омск, микрорайон Городок Нефтяников, переулок Энтузиастов, 2", 
 6622.0: "Россия, Курск, проспект Победы, 48", 
 6638.0: "улица Карла Маркса, 23, жилой район Адлер, Сочи, Краснодарский край", 
 6654.0: "посёлок городского типа Рощинский, 19А, городское поселение Рощинский, Волжский район, Самарская область",
 6655.0: "посёлок городского типа Рощинский, 19А, городское поселение Рощинский, Волжский район, Самарская область", 
 6668.0: "Россия, Омск, Сибирский проспект, 7", 
 6706.0: "Россия, Тула, Одоевское шоссе, 98", 
 6710.0: "Россия, Волгоградская область, Волжский, улица Мира, 74", 
 6752.0: "Россия, Курск, улица Карла Маркса, 10", 
 6778.0: "Россия, Краснодарский край, Тимашевск, Братская улица, 166", 
 6790.0: "улица Кирова, 58, Сочи, Краснодарский край", 
 6792.0: "улица Кирова, 58, Сочи, Краснодарский край", 
 6818.0: "Россия, Московская область, Истра, улица Ленина, 80А",
 6826.0: "улица Ленина, 27А, рабочий посёлок Чик, Коченёвский муниципальный район, Новосибирская область", 
 6856.0: "Россия, Московская область, Павловский Посад, улица Кирова, 4", 
 6857.0: "Россия, Московская область, Павловский Посад, улица Кирова, 4", 
 6880.0: "Россия, Великий Новгород, улица Ломоносова, 15", 
 6885.0: "Россия, Великий Новгород, улица Ломоносова, 15", 
 6919.0: "Россия, Волгоградская область, Волжский, проспект имени Ленина, 75", 
 6965.0: "Россия, Псков, улица Труда, 54", 
 6969.0: "Садовая улица, 3, Пионерский, Калининградская область", 
 7005.0: "Россия, Тула, Путейская улица, 5",
 7015.0: "Россия, Брянск, улица Тельмана, 68Б", 
 7017.0: "Россия, Калужская область, Людиново, улица Маяковского, 4А", 
 7024.0: "Россия, Тула, улица Сойфера, 18", 
 7026.0: "Россия, Тула, Путейская улица, 4", 
 7029.0: "Россия, Краснодарский край, Сочи, микрорайон Центральный, улица Горького, 56", 
 7030.0: "улица Карла Маркса, 23, жилой район Адлер, Сочи, Краснодарский край", 
 7032.0: "Россия, Краснодарский край, Сочи, микрорайон Центральный, улица Горького, 56", 
 7042.0: "Россия, Республика Хакасия, Абакан, Торговая улица, 5Вс2", 
 7068.0: "Россия, Кемеровская область — Кузбасс, Ленинск-Кузнецкий, улица Гагарина, 19",
 7089.0: "Россия, Киров, улица Ленина, 20", 
 7091.0: "Россия, Омск, улица Химиков, 12Б", 
 7112.0: "Россия, Тула, улица Кирова, 20", 
 7115.0: "Россия, Тула, Октябрьская улица, 91", 
 7137.0: "Россия, Республика Татарстан (Татарстан), Казань, Московская улица, 2", 
 7142.0: "Россия, Калининград, Киевская улица, 119Б", 
 7169.0: "Россия, Республика Саха (Якутия), Ленск, улица Дзержинского, 30", 
 7171.0: "Россия, Республика Саха (Якутия), городской округ Якутск, село Маган, улица 40 лет Победы 6",
 7173.0: "Россия, Республика Саха (Якутия), Мирный, улица Солдатова, 5А", 
 7181.0: "Россия, Белгород, улица Победы, 83Б", 
 7191.0: "Россия, Забайкальский край, Чита, улица Строителей, 1", 
 7271.0: "Россия, Омск, проспект Мира, 11", 
 7320.0: "Россия, Москва, Театральный проезд, 5с2", 
 7321.0: "Россия, Москва, Театральный проезд, 5с2", 
 7322.0: "Россия, Москва, Театральный проезд, 5с2", 
 7349.0: "Вокзальная улица, 52, Южно-Сахалинск, Сахалинская область",
 7357.0: "Россия, Алтайский край, Барнаул, микрорайон ВРЗ, улица Воровского, 108В", 
 7362.0: "Россия, Алтайский край, Барнаул, проспект Ленина, 102В", 
 7389.0: "Россия, Пермь, Парковый проспект, 17", 
 7395.0: "Россия, Краснодарский край, городской округ Сочи, село Весёлое, улица Мира, 2Б", 
 7396.0: "Россия, Краснодарский край, Сочи, посёлок Лоо, Азовская улица, 7", 
 7442.0: "улица Ленина, 10, рабочий посёлок Марьяновка, Омская область", 
 7464.0: "Россия, Красноярск, улица Ломоносова, 47", 
 7480.0: "Россия, Нижний Новгород, Московское шоссе, 17А", 
 7519.0: "Россия, Нижний Новгород, улица Октябрьской Революции, 61А",
 7523.0: "Россия, Республика Башкортостан, Уфа, улица Энтузиастов, 20", 
 7534.0: "Россия, Ростовская область, Батайск, Северная улица, 4", 
 7552.0: "Россия, Амурская область, Благовещенск, Станционная улица, 70", 
 7604.0: "Россия, Хабаровский край, Комсомольск-на-Амуре, проспект Мира, 37", 
 7638.0: "Россия, Самара, посёлок Соцгород, Долотный переулок, 16", 
 7640.0: "1-й микрорайон, 11, Чита, Забайкальский край", 
 7656.0: "Россия, Забайкальский край, Чита, улица Курнатовского, 25", 
 7664.0: "Россия, Амурская область, Тындинский муниципальный округ, посёлок Юктали, Привокзальная улица, 3", 
 7670.0: "Советская улица, 47, село Соловьёвск, Тындинский муниципальный округ, Амурская область",
 7672.0: "Россия, Владимирская область, Александров, Вокзальная улица, 10", 
 7682.0: "Россия, Иркутская область, Зима, улица Ленина, 5", 
 7725.0: "Россия, Саратовская область, Энгельс, улица Колотилова, 86", 
 7726.0: "Россия, Республика Бурятия, Улан-Удэ, Норильская улица, 12Б", 
 7743.0: "Шоссейная улица, 10, посёлок Северный, муниципальный округ Истра, Московская область", 
 7753.0: "Россия, Воронеж, улица 9 Января, 292Б", 
 7768.0: "Россия, Воронежская область, Лиски, проспект Ленина, 43", 
 7769.0: "Россия, Липецкая область, Елец, улица Орджоникидзе, 40", 
 7770.0: "Россия, Иркутск, микрорайон Первомайский, улица Вампилова, 2/2",
 7778.0: "Россия, Белгород, улица Белгородского Полка, 34", 
 7795.0: "Россия, Липецкая область, Елец, улица Пушкина, 22", 
 7819.0: "Вокзальная улица, 11, городской посёлок Зелёный Бор, Минусинский муниципальный округ, Красноярский край,", 
 7833.0: "Россия, Томск, проспект Мира, 30", 
 7836.0: "Россия, Ханты-Мансийский автономный округ — Югра, Сургут, Югорский тракт, 38", 
 7849.0: "улица Мусы Джалиля, 6, посёлок городского типа Беркакит, Нерюнгринский район, Республика Саха (Якутия)", 
 7850.0: "Россия, Нижний Новгород, улица Веденяпина, 13", 
 7851.0: "Россия, Нижний Новгород, улица Веденяпина, 13", 
 7860.0: "Россия, Нижний Новгород, улица Веденяпина, 13",
 7865.0: "Россия, Сахалинская область, Южно-Сахалинск, улица Ленина, 234", 
 7868.0: "Россия, Сахалинская область, Южно-Сахалинск, улица Ленина, 234", 
 7928.0: "Россия, Республика Татарстан (Татарстан), Казань, Вахитовский район, улица Калинина, 62", 
 7935.0: "Россия, Республика Татарстан (Татарстан), Казань, улица Максимова, 3", 
 7959.0: "Россия, Самарская область, Кинель, улица Маяковского, 84А", 
 7963.0: "Россия, Пермь, улица Мира, 26", 
 7966.0: "микрорайон ИКЕА, к1, Химки, Московская область", 
 7989.0: "Россия, Пермь, улица Мира, 26А", 
 8070.0: "Россия, Волгоградская область, Волжский, бульвар Профсоюзов, 1Б",
 8079.0: "Россия, Киров, улица Чапаева, 55", 
 8093.0: "Россия, Нижний Новгород, проспект Гагарина, 105", 
 8097.0: "Россия, Сахалинская область, Холмск, площадь Ленина, 9", 
 8098.0: "Россия, Сахалинская область, Южно-Сахалинск, улица Ленина, 234", 
 8126.0: "Россия, Великий Новгород, Октябрьская улица, 5", 
 8134.0: "Россия, Пермь, улица Революции, 26", 
 8135.0: "Россия, Пермь, улица Революции, 26", 
 8171.0: "Россия, Тула, улица Максима Горького, 1Б", 
 8181.0: "Россия, Воронежская область, Рамонский район, Яменское сельское поселение, посёлок Солнечный, Парковая улица, 3",
 8220.0: "Россия, Киров, улица Воровского, 77", 
 8229.0: "улица Маяковского, 30, микрорайон Юбилейный, Королёв, Московская область", 
 8254.0: "Россия, Краснодарский край, Сочи, микрорайон Центральный, Московская улица, 5", 
 8256.0: "Россия, Краснодарский край, Сочи, микрорайон Центральный, Московская улица, 5", 
 8273.0: "Россия, Амурская область, Благовещенск, Игнатьевское шоссе, 15", 
 8282.0: "Россия, Республика Башкортостан, Уфа, Первомайская улица, 44", 
 8283.0: "Россия, Республика Башкортостан, Уфа, Первомайская улица, 44", 
 8284.0: "Россия, Республика Башкортостан, Уфа, Первомайская улица, 44", 
 8293.0: "Россия, Тульская область, Новомосковск, улица Дружбы, 1/24",
 8306.0: "микрорайон ИКЕА, к1, Химки, Московская область", 
 8322.0: "Россия, Ярославская область, Ростов Великий, 1-й микрорайон, 34", 
 8331.0: "Россия, Камчатский край, Петропавловск-Камчатский, проспект Рыбаков, 2/1", 
 8342.0: "Россия, Свердловская область, Екатеринбург, проспект Ленина, 101", 
 8343.0: "Россия, Свердловская область, Екатеринбург, проспект Ленина, 101", 
 8344.0: "Россия, Приморский край, посёлок городского типа Пограничный, улица Ленина, 133А", 
 8369.0: "Россия, Нижний Новгород, проспект Гагарина, 162А", 
 8370.0: "Россия, Нижний Новгород, проспект Гагарина, 162А", 
 8391.0: "Россия, Московская область, Одинцовский городской округ, посёлок городского типа Лесной Городок, Школьная улица, 1",
 8431.0: "Россия, Сахалинская область, Южно-Сахалинск, проспект Мира, 66", 
 8432.0: "Россия, Сахалинская область, Южно-Сахалинск, Сахалинская улица, 28", 
 8435.0: "Россия, Алтайский край, Бийск, улица Максима Горького, 69", 
 8507.0: "Россия, Ульяновск, Хрустальная улица, 19Б", 
 8521.0: "Россия, Республика Тыва, Эрзинский кожуун, село Эрзин, Комсомольская улица, 19", 
 8610.0: "Россия, Ульяновск, улица Железной Дивизии, 5Б", 
 8627.0: "Россия, Москва, улица Арбат, 1", 
 8630.0: "Россия, Ульяновск, улица Богдана Хмельницкого, 19Б", 
 8633.0: "Россия, Ростовская область, Каменск-Шахтинский, улица Щаденко, 70",
 8646.0: "Россия, Москва, Ломоносовский проспект, 25к1", 
 8652.0: "улица Пушкина, 65/2, Томск", 
 8661.0: "Россия, Республика Бурятия, Тарбагатайский район, сельское поселение Тарбагатайское, село Тарбагатай, улица Ленина, 31", 
 8668.0: "Россия, Республика Дагестан, Хасавюрт, улица Омар-Гаджи Шахтаманова, 3", 
 8672.0: "Россия, Самарская область, сельское поселение Красный Яр, село Красный Яр, Комсомольская улица, 86", 
 8698.0: "Россия, Республика Тыва, Каа-Хемский кожуун, село Сарыг-Сеп, Енисейская улица, 143", 
 8729.0: "Россия, Брянская область, рабочий посёлок Дубровка, улица Ленина, 91", 
 8748.0: "Россия, Томская область, Шегарское сельское поселение, село Мельниково, Московская улица, 9/8", 
 8782.0: "Россия, Приморский край, Ханкайский муниципальный округ, село Камень-Рыболов, Октябрьская улица, 10А",
 8800.0: "Россия, Республика Калмыкия, село Малые Дербеты, Советская улица, 13", 
 8805.0: "улица Пушкина, 59, село Троицкое, Троицкий район, Алтайский край", 
 8806.0: "улица Чкалова, 14, село Троицкое, Целинный район, Республика Калмыкия",
 1681.0: "проспект 40-летия Победы, 85, Ростов-на-Дону",
 1032.0: "микрорайон 70-летия ВЛКСМ, 16/1, Северный жилой район, Оренбург",
 2611.0: "Красноармейский проспект, 11, Тула",
 4728.0: "улица Фрунзе, 56, Нижний Тагил, Свердловская область",
 4816.0: "улица 8 Марта, 9А, Октябрьский, Республика Башкортостан",
 4997.0: "улица имени Газеты Пионерская Правда, 2, Красноярск,",
 5025.0: "улица Малиновского, 23Д, Ростов-на-Дону",
 5115.0: "улица Одоевского, 29, Санкт-Петербург",
 5176.0: "Советская улица, 194к2, Тамбов",
 5212.0: "Россия, Ярославская область, Рыбинск, проспект Ленина, 154В",
 5224.0: "улица Одоевского, 29А, Санкт-Петербург",
 5348.0: "Телевизорная улица, 1с78, Красноярск",
 5413.0: "проспект Тракторостроителей, 1/34, Чебоксары, Чувашская Республика — Чувашия",
 5528.0: "улица Трудовые Резервы, 33А, Новомосковск, Тульская область",
 6653.0: "Студенческая улица, 4Б, Астрахань",
 6724.0: "проспект Большевиков, 21, Санкт-Петербург,",
 6725.0: "проспект Большевиков, 21, Санкт-Петербург,",
 6787.0: "улица 60 лет СССР, 10, Фрязино, Московская область",
 6840.0: "Новоугличское шоссе, 53А, Сергиев Посад, Московская область,",
 6878.0: "улица Александра Матросова, 7, Красноярск",
 6879.0: "улица Александра Матросова, 7, Красноярск",
 6895.0: "улица Бутлерова, 40, Санкт-Петербург",
 6896.0: "улица Бутлерова, 40, Санкт-Петербург",
 7159.0: "проспект Юных Пионеров, 146, Самара",
 7193.0: "Владимирская улица, 141с1, Балашиха, Московская область",
 7428.0: "улица Гагарина, 2, Лесозаводск, Приморский край",
 7561.0: "улица Александра Суворова, 40, Калининград",
 7832.0: "улица Александра Матросова, 7, Красноярск",
 7846.0: "улица Гагарина, 3, Омск",
 8012.0: "улица 20-летия Октября, 90А, Воронеж",
 8013.0: "улица 20-летия Октября, 90А, Воронеж",
 8014.0: "улица 20-летия Октября, 90А, Воронеж",
 8015.0: "улица 20-летия Октября, 90А, Воронеж",
 8044.0: "Омская улица, 2К, Ростов-на-Дону",
 8047.0: "проспект Тракторостроителей, 1/34, Чебоксары, Чувашская Республика — Чувашия",
 3011.0: "2-я Советская улица, 17, Санкт-Петербург",
 3901.0: "Россия, Красноярск, проспект Мира, 82",
 4105.0: "улица 50-летия ВЛКСМ, 4Г, Королёв, Московская область",
 1793.0: "проспект Просвещения, 84к1, Санкт-Петербург",
 1101.0: "улица Героев Сибиряков, 65А, Воронеж",
 1190.0: "проспект 25 Октября, 42Д, Гатчина, Ленинградская область,",
 2510.0: "Варшавское шоссе, 9, Москва",
 2521.0: "Газетная улица, 95, Нижний Тагил, Свердловская область",
 2621.0: "проспект Обуховской Обороны, 305, Санкт-Петербург",
 2755.0: "улица Богдана Хмельницкого, 214, Омск",
 8405.0: "улица Дыбенко, 26, Санкт-Петербург",
 4786.0: "Комсомольская улица, 295, Южно-Сахалинск, Сахалинская область",
 5283.0: "улица Ижорского Батальона, 25, Псков",
 5287.0: "улица Шамиля Усманова, 38А, Набережные Челны, Республика Татарстан (Татарстан)",
 5428.0: "улица Солнечная Поляна, 15, Барнаул, Алтайский край",
 5792.0: "Коммунистическая улица, 39, Новокуйбышевск, Самарская область",
 5793.0: "Коммунистическая улица, 39, Новокуйбышевск, Самарская область",
 5810.0: "Коммунистическая улица, 10, село Поспелиха, Поспелихинский Центральный сельсовет, Поспелихинский район, Алтайский край",
 5985.0: "Тульское шоссе, 4Б, Ефремов, Тульская область",
 6017.0: "улица Пушкина, 1, Советская Гавань, Хабаровский край",
 6094.0: "Коммунистическая улица, 39, Новокуйбышевск, Самарская область",
 6095.0: "Коммунистическая улица, 39, Новокуйбышевск, Самарская область",
 6211.0: "улица Декабристов, 185, Казань, Республика Татарстан (Татарстан)",
 6276.0: "Таганрогская улица, 104, Ростов-на-Дону",
 6277.0: "Таганрогская улица, 104, Ростов-на-Дону",
 6278.0: "Таганрогская улица, 104, Ростов-на-Дону",
 6279.0: "Таганрогская улица, 104, Ростов-на-Дону",
 6713.0: "Советская улица, 28, Нововятский район, Киров",
 7150.0: "проспект Королёва, 29, Ростов-на-Дону",
 7198.0: "Краснознамённая улица, 224А, Уссурийск, Приморский край",
 7330.0: "улица Героя Суворова, 1, посёлок городского типа Новобурейский, Бурейский муниципальный округ, Амурская область",
 7758.0: "площадь Генерала Черняховского, 1, Центральный район, городской округ Воронеж",
 7788.0: "Байкальская улица, 265Б, Иркутск",
 7790.0: "Железнодорожная улица, 6, рабочий посёлок Таловая, Воронежская область",
 2921.0: "улица 8 Марта, 32, Иваново",
 1755.0: "Высотная улица, 2Г, Красноярск",
 1756.0: "Высотная улица, 2Г, Красноярск",
 1757.0: "Высотная улица, 2Г, Красноярск",   
 1113.0: "улица Ленинского Комсомола, 21А, Чебоксары, Чувашская Республика — Чувашия",   
 1340.0: "Ленинградский проспект, 28В, Кемерово",   
 1470.0: "Московское шоссе, 65А, Рязань",   
 2285.0: "проспект Маршала Жукова, 5/1, Иркутск",   
 2497.0: "Варшавское шоссе, 97, Москва",   
 2667.0: "улица Олега Кошевого, 1В, Владивосток, Приморский край",   
 2810.0: "проспект Мира, 52, Красноярск",   
 2815.0: "Золотая Нива метро новосибирск",   
 8417.0: "проезд Мира, 15, Железногорск, Красноярский край",   
 8464.0: "Северная улица, 324А, Краснодар",   
 4935.0: "Почтовая улица, 12, Куровское, Орехово-Зуевский городской округ, Московская область",
 4959.0: "Московское шоссе, 94В, Ульяновск",   
 5014.0: "улица Верхняя Дуброва, 36А, Владимир",
 5032.0: "улица Владимира Высоцкого, 25с3, Томск",
 5111.0: "Привокзальная площадь, 6, Мурино, Всеволожский район, Ленинградская область",
 5116.0: "Московский проспект, 71, Санкт-Петербург",
 5119.0: "Слуцкая улица, 3А, Павловск, Пушкинский район, Санкт-Петербург",
 5332.0: "улица Сидоренко, 1с2, Находка, Приморский край",
 5362.0: "улица 10 лет Октября, 109, Омск",
 5406.0: "улица Гагарина, 30, Ульяновск",
 5459.0: "улица Луначарского, 28, Пенза",
 5605.0: "Красногвардейская улица, 35, Комсомольск-на-Амуре, Хабаровский край",
 5728.0: "улица Мира, 284/1, Ставрополь",
 5729.0: "улица Мира, 284/1, Ставрополь",
 5730.0: "улица Мира, 284/1, Ставрополь",
 5732.0: "улица Пермякова, 78А, Тюмень",
 5733.0: "улица Пермякова, 78А, Тюмень",
 5734.0: "улица Пермякова, 78А, Тюмень",
 6119.0: "улица имени Н.И. Вавилова, 38/114, Саратов",
 6608.0: "улица 30 лет Победы, 156, Балашов, Саратовская область",
 7007.0: "улица Луначарского, 67А, Тула",
 7056.0: "улица 9 Мая, 13Б, Рыбинск, Ярославская область",
 7095.0: "Комсомольская улица, 63, рабочий посёлок Москаленки, Омская область",
 7129.0: "Привокзальная площадь, 6, Мурино, Всеволожский район, Ленинградская область",
 7654.0: "улица Дикопольцева, 29/6, Комсомольск-на-Амуре, Хабаровский край,",
 7671.0: "посёлок Муртыгит, Тындинский муниципальный округ, Амурская область",
 7707.0: "улица Мира, 2, Тюмень",
 7799.0: "улица Ползунова, 15к1, Новосибирск",
 7913.0: "улица Ким Ю Чена, 15, Хабаровск",
 7914.0: "улица Ким Ю Чена, 15, Хабаровск",
 7945.0: "Коммунистическая улица, 17, Лиски, Воронежская область",
 7946.0: "Коммунистическая улица, 17, Лиски, Воронежская область",
 8020.0: "улица Кирова, 10/25, Ярославль",
 8032.0: "Коммунистическая улица, 17, Лиски, Воронежская область",
 8154.0: "улица Воинов-Интернационалистов, 18, Железногорск",
 8225.0: "улица имени Н.И. Вавилова, 38/114, Саратов",
 8600.0: "Варшавское шоссе, 74к2, Москва",
 491.0: "улица Генерала Штеменко, 7, Волгоград",
 934.0: "Советский проспект, 2Б, Ивантеевка, городской округ Пушкинский, Московская область",
 1072.0: "Аэродромная улица, 47А, Самара",
 1151.0: "улица Мира, 15, Железногорск, Курская область",
 2589.0: "Козловская улица, 25, Волгоград",
 2597.0: "улица Революции, 50, Ялуторовск, Тюменская область",
 2686.0: "Козловская улица, 25, Волгоград",
 4765.0: "Металлургическая улица, 102/2, микрорайон Сельмаш, Ростов-на-Дону",
 4973.0: "улица Менжинского, 2, Ростов-на-Дону",
 5038.0: "улица Кирова, 19, Старая Купавна, Богородский городской округ, Московская область",
 5189.0: "проспект Металлистов, 115, Санкт-Петербург",
 5261.0: "Светлоярская улица, 24, Нижний Новгород",
 5262.0: "улица Дьяконова, 11А, Нижний Новгород",
 5272.0: "улица Карла Маркса, 6с7, Клин, Московская область",
 5344.0: "улица Кирова, 23, Тула",
 5417.0: "проспект Текстильщиков, 19, Ленинск-Кузнецкий, Кемеровская область — Кузбасс",
 5482.0: "микрорайон 8А, 7А, Нефтеюганск, Ханты-Мансийский автономный округ — Югра",
 5540.0: "Кирпичная улица, 33, Москва",
 5548.0: "площадь 40 лет Сталинградской Битвы, 7, рабочий посёлок Городище, Волгоградская область",
 5558.0: "улица Мира, 3Д, Сергиев Посад, Московская область",
 6541.0: "улица Борисовские Пруды, 26к2, Москва",
 6554.0: "улица Фучика, 2А, Санкт-Петербург",
 6711.0: "улица Декабрьских Событий, 85, Иркутск,",
 7077.0: "улица 50 лет Октября, 154, Благовещенск, Амурская область",
 7106.0: "улица Профессора Баранова, 34, Калининград",
 7216.0: "улица 9 Мая, 59А, Красноярск",
 7260.0: "микрорайон Луговой, 4, посёлок городского типа Ясногорск, Оловяннинский муниципальный округ, Забайкальский край",
 7422.0: "улица Карла Маркса, 13Ак3, Ульяновск",
 7625.0: "улица Зои Космодемьянской, 13, Ртищево, Саратовская область",
 7784.0: "улица Патриса Лумумбы, 4Б, Екатеринбург, Свердловская область",
 7948.0: "улица Труфанова, 19, Ярославль",
 7967.0: "Кольский проспект, 158к1, Мурманск",
 7975.0: "улица Труфанова, 19, Ярославль",
 7993.0: "Комитетская улица, 56/64, Новочеркасск, Ростовская область",
 7995.0: "Комитетская улица, 56/64, Новочеркасск, Ростовская область",
 8017.0: "Северная улица, 357, Краснодар",
 8030.0: "Северная улица, 357, Краснодар",
 8166.0: "улица Руставели, 43, Санкт-Петербург",
 8214.0: "Комсомольская улица, 34, Новомосковск, Тульская область",
 8253.0: "Комсомольская улица, 34, Новомосковск, Тульская область",
 8356.0: "улица Митрофана Седина, 62, микрорайон Центральный, Краснодар,",
 4194.0: "проспект Мира, 19, Красноярск",
 2019.0: "улица Волкова, 16Б, Новомичуринск, Пронский район, Рязанская область",
 910.0: "Дмитровское шоссе, 13А, Москва",
 912.0: "Дмитровское шоссе, 13А, Москва",
 1689.0: "Звенигородская улица, 1к2, Санкт-Петербург",
 5112.0: "Московский проспект, 28, Санкт-Петербург",
 5113.0: "Московский проспект, 28, Санкт-Петербург",
 5117.0: "Невский проспект, 2, Санкт-Петербург",
 5241.0 : "проспект Ленина, 8Г, Мончегорск, Мурманская область",
 5371.0: "улица Юлиуса Фучика, 90, Казань, Республика Татарстан (Татарстан),",
 5467.0: "улица Страж Революции, 7/5, Нижний Новгород",
 5539.0: "Артиллерийская улица, 1, Санкт-Петербург",
 5563.0: "Новая улица, 6, Зеленокумск, Советский муниципальный округ, Ставропольский край",
 5583.0: "улица Петра Алексеева, 76, Якутск, Республика Саха (Якутия)",
 5637.0: "улица Ключ-Камышенское Плато, 14/1, Новосибирск",
 5684.0: "Полевая улица, 9, Самара",
 5941.0: "улица Свободы, 26, Ярославль",
 5942.0: "улица Свободы, 26, Ярославль",
 5943.0: "улица Свободы, 26, Ярославль",
 5944.0: "улица Свободы, 26, Ярославль",
 5976.0: "улица 40 лет Комсомола, 6, Новосибирск",
 6665.0: "Архангельская улица, 16, Нижний Новгород",
 6888.0: "улица Володарского, 5А, Сестрорецк, Санкт-Петербург",
 6889.0: "улица Володарского, 5А, Сестрорецк, Санкт-Петербург",
 6966.0: "улица Карла Либкнехта, 10, микрорайон Центральный, Сочи, Краснодарский край,",
 6967.0: "улица Карла Либкнехта, 10, микрорайон Центральный, Сочи, Краснодарский край,",
 7132.0: "Московский проспект, 28, Санкт-Петербург",
 7152.0: "Боткинская улица, 5, Санкт-Петербург",
 7248.0: "улица 50 лет Октября, 20, Благовещенск, Амурская область",
 7373.0: "Змеиногорский тракт, 71В, Барнаул, Алтайский край",
 7474.0: "улица Станция Семёнов, 1, Семёнов, Нижегородская область",
 7548.0: "Благовещенская улица, 15, Белогорск, Амурская область",
 7620.0: "Россия, Самара, метро Московская",
 7693.0: "улица Январских Событий, 36Ж, Зима, Иркутская область",
 7709.0: "улица Энергетиков, 14, посёлок городского типа Ясногорск, Оловяннинский муниципальный округ, Забайкальский край",
 7896.0: "улица Богдана Хмельницкого, 17, Новосибирск",
 7897.0: "улица Богдана Хмельницкого, 17, Новосибирск",
 7898.0: "улица Богдана Хмельницкого, 17, Новосибирск",
 7922.0: "Днепропетровский проезд, 4Ас31А, Москва",
 7934.0: "Днепропетровский проезд, 4Ас31А, Москва",
 8280.0: "улица Гагарина, 21В, Невинномысск, Ставропольский край",
 8281.0: "улица Гагарина, 21В, Невинномысск, Ставропольский край",
 3831.0: "проспект Мира, 110, Красноярск",
 4324.0: "улица Тувинских Добровольцев, 23, Кызыл, Республика Тыва",
 4360.0: "Центральная улица, 9, поселок Суходольск, Самарское сельское поселение, Азовский район, Ростовская область",
 4361.0: "проспект Кирова, 53, Ленинск-Кузнецкий, Кемеровская область — Кузбасс,",
 7153.0: "Сенная площадь, 1, Санкт-Петербург",
 5401.0: "улица Гагарина, 3А, Туапсе, Краснодарский край",
 1769.0: "проспект Просвещения, 19, Санкт-Петербург",
 3555.0: "Интернациональная улица, 117, Кызыл, Республика Тыва",
 4971.0: "улица Адмирала Нахимова, 125, Астрахань",
 6712.0: "1-й микрорайон, 62, Шелехов, Иркутская область",
 5349.0: "улица Александра Матросова, 30/1, Красноярск",
 6549.0: "Привольная улица, 65/32, Москва",
 8036.0: "улица Карла Маркса, 148А, Красноярск,",
 6012.0: "улица Рождественского, 6к1, Омск",
 6391.0: "Новая улица, 4, Оренбург",
 6550.0: "Привольная улица, 65/32, Москва",
 7843.0: "улица Карла Маркса, 148А, Красноярск",
 5600.0: "Политехническая улица, 86, Благовещенск, Амурская область",
 1248.0: "4-й проезд, 57А, Самара",
 3131.0: "площадь МОПРа, 9, Центральный район, Челябинский городской округ,",
 8407.0: "проспект Стачек, 90к2, Санкт-Петербург",
 1084.0: "улица Максима Горького, 285, Батайск, Ростовская область",
 6661.0: "1-й микрорайон, 45, Шелехов, Иркутская область",
 6390.0: "Новая улица, 4, Оренбург",
 7912.0: "улица 70 лет Октября, 43, посёлок городского типа Таксимо, Муйский муниципальный округ, Республика Бурятия,",
 1684.0: "проспект Просвещения, 19, Санкт-Петербург",
 1230.0: "улица 8 Марта, 19, Орёл",
 7968.0: "улица Карла Маркса, 148А, Красноярск",
 5532.0: "улица Мира, 3Д, Новомосковск, Тульская область",
 7319.0: "улица Пионеров КАТЭКа, 7А, городской посёлок Дубинино, Шарыповский муниципальный округ, Красноярский край",
 4967.0: "улица Николая Островского, 146А, Астрахань",
 1238.0: "улица Мира, 33, посёлок Дербышки, Советский район, Казань, Республика Татарстан (Татарстан)",
 6552.0: "Привольная улица, 65/32, Москва",
 4766.0: "улица Доватора, 267, Ростов-на-Дону",
 1351.0: "Южное шоссе, 5, Самара",
 7990.0: "улица 20-летия Победы, 173В, Соликамск, Пермский край",
 7206.0: "улица Парижской Коммуны, 51, Малоярославец, Калужская область",
 5073.0: "Торжковская улица, 5, Санкт-Петербург",
 5068.0: "улица Королёва, 2Ак3, Белгород",
 7299.0: "улица Ублинские Горы, 1А, Старый Оскол, Белгородская область",
 2782.0: "8-й микрорайон, 10, Назарово, Красноярский край",
 7821.0: "Водопроводная улица, 38, Тюмень",
 7377.0: "улица Фатыха Амирхана, 49, Казань, Республика Татарстан (Татарстан)",
 2780.0: "улица Красной Армии, 121, Красноярск",
 6073.0: "Полковая улица, 1/25, Пушкин, Санкт-Петербург,",
 6807.0: "улица Николая Островского, 146А, Астрахань",
 694.0: "Автозаводская улица, 13/1, Москва",
 1068.0: "улица Пушкина, 102, Бор, Нижегородская область",
 4139.0: "проспект Чернышевского, 17, Санкт-Петербург",
 2441.0: "Россия, Приморский край, Большой Камень, улица Академика Курчатова 36",
 6153.0: "улица 50 лет Октября, 19, Алексин, Тульская область,",
 3313.0: "площадь Мира, 2Б, Центральный район, городской округ Красноярск",
 5147.0: "Комсомольский проспект, 58А, Усолье-Сибирское, Иркутская область",
 6018.0: "улица 70 лет Октября, 43, посёлок городского типа Таксимо, Муйский муниципальный округ, Республика Бурятия",
 8536.0: "улица Дружба, 25, Ак-Довурак, Республика Тыва",
 7225.0: "Гвардейская улица, 31/42, Казань, Республика Татарстан (Татарстан)",
 1054.0: "улица Герцена, 3А, Альметьевск, Республика Татарстан (Татарстан)",
 1527.0: "улица Ленина, 15П, Нижневартовск, Ханты-Мансийский автономный округ — Югра",
 7096.0: "24-я Северная улица, 168/2, Омск",
 6392.0: "Новая улица, 4, Оренбург",
 3060.0: "проспект Мира, 80/28, Красноярск",
 8530.0: "улица А.О. Емельянова, 19А, Южно-Сахалинск, Сахалинская область",
 6551.0: "Привольная улица, 65/32, Москва",
 2794.0: "8-й микрорайон, 2А, Ачинск, Красноярский край",
 6054.0: "проспект Гагарина, 48/3, Оренбург",
 7090.0: "Набережная улица, 10, Владивосток, Приморский край",
 6389.0: "Новая улица, 4, Оренбург,",
 2097.0: "улица Чкалова, 1, Оренбург",
 7151.0: "проспект Шолохова, 128, Ростов-на-Дону",
 2792.0: "проспект Мира, 10, Красноярск",
 223.0: "шоссе Космонавтов, 65, Пермь",
 7098.0: "улица Муравьёва-Амурского, 32, Хабаровск",
 5027.0: "набережная реки Фонтанки, 203ДК, Санкт-Петербург",
 5598.0: "улица 50 лет Октября, 15, Благовещенск, Амурская область"}

# В списке to_drop_ids лежает строки, которые не удалось обрабоать вручную
to_drop_ids = [498.0, 836.0, 837.0, 838.0, 1688.0, 1830.0, 1989.0, 2155.0, 2489.0, 2560.0, 2561.0, 2562.0, 2659.0, 2761.0, 2981.0, 3110.0, 3303.0, 3440.0, 3751.0, 3848.0,
 3890.0, 4188.0, 4855.0, 4921.0, 5095.0, 5281.0, 5350.0, 5530.0, 5580.0, 7304.0, 7366.0, 7598.0, 7626.0, 8091.0, 8436.0, 3393.0, 7698.0, 5188.0, 6014.0, 6125.0, 6367.0, 
 2236.0, 5001.0, 2236.0, 5001.0, 7614.0, 3315.0, 406.0, 7286.0, 3504.0, 8565.0, 3127.0, 1666.0, 1528.0, 2453.0, 2605.0, 3364.0, 6145.0, 6146.0, 6147.0, 7111.0, 7302.0, 8798.0,
 1866.0, 8412.0, 4814.0, 5054.0, 5060.0, 1892.0, 5568.0, 7859.0, 7864.0, 8377.0, 468.0, 8176.0, 6890.0, 3586.0, 2439.0, 3739.0, 8082.0]

# Удаляем строки, которые невозможно восстановить
all_data = all_data[~all_data["id"].isin(to_drop_ids)]

# Обновляем address_cleaned по id
for idx, new_addr in updates.items():
    all_data.loc[all_data["id"] == idx, "address_rus"] = new_addr

# Выбираем строки по id из updates
rows = all_data[all_data["id"].isin(updates.keys())].iterrows()

# Геокодируем выбранные строки
all_data = process_geocoding(df=all_data, rows=rows, api_key=API_KEY, use_en=False, use_ru=True)

# Сохраняем обновлённый датафрейм
all_data.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')

# Правило: выбираем строки, где id присутствует в словаре updates
rule_updates = all_data["id"].isin(updates.keys())

# Разделяем данные по условию rule_updates для обновления адресов
df_left_to_clean = split_to_clean_and_left(df_left_to_clean, rule_updates, source_suffix="en")

# Выводим оставшиеся строки после разделения - пустой датасет (просто для проверки)
display(df_left_to_clean)

,id,atm_group,address,address_rus,geo_address_en,geo_address_ru,geo_lon_en,geo_lon_ru,geo_lat_en,geo_lat_ru,...,area_en,area_ru,locality_en,locality_ru,street_en,street_ru,house_en,house_ru,target,isTrain


In [ ]:
# Чтение файла
final_df = pd.read_csv("data/cleaned_data.csv", encoding="utf-8")

# Отображение датасета
display(final_df)

# Сохранение финальной версии исправленного датасета
final_df.to_csv("data/geo_data_cleaned.csv", index=False, encoding="utf-8")

,id,atm_group,address,geo_address,geo_lon,geo_lat,country,province,area,locality,street,house,target,isTrain
0,1.0,32.0,"MARATA, 17 S.PETERBURG","Россия, Санкт-Петербург, улица Марата, 17",30.353382,59.928916,Россия,Санкт-Петербург,NaN,Санкт-Петербург,улица Марата,17,NaN,False
1,2.0,32.0,"TYUSHINA, 11 ST.PETERSBURG","Россия, Санкт-Петербург, улица Тюшина, 11",30.346887,59.917619,Россия,Санкт-Петербург,NaN,Санкт-Петербург,улица Тюшина,11,NaN,False
2,3.0,1022.0,ABB 5 M.DZHALILYA N.SHIGALEEVO,"Россия, Республика Татарстан (Татарстан), Пест...",49.443201,55.807779,Россия,Республика Татарстан (Татарстан),Шигалеевское сельское поселение,село Новое Шигалеево,улица Мусы Джалиля,5,NaN,False
3,4.0,496.5,PUSHKINA 15 ELISTA,"Россия, Республика Калмыкия, Элиста, улица А.С...",44.270019,46.309498,Россия,Республика Калмыкия,городской округ Элиста,Элиста,улица А.С. Пушкина,15,NaN,False
4,5.0,496.5,BUDENNOGO 7A ELISTA,"Россия, Республика Калмыкия, Элиста, улица С.М...",44.260605,46.318231,Россия,Республика Калмыкия,городской округ Элиста,Элиста,улица С.М. Будённого,7А,0.019958,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8677,8807.0,496.5,OKTYABRSKAYA 18A IKI-BURUL,"Россия, Республика Калмыкия, посёлок Ики-Бурул...",44.647851,45.826910,Россия,Республика Калмыкия,Ики-Бурульский район,посёлок Ики-Бурул,Октябрьская улица,16,-0.090848,True
8678,8808.0,496.5,REVOLUTIONNAYA 1 LAGAN,"Россия, Республика Калмыкия, Лагань, Революцио...",47.377247,45.400131,Россия,Республика Калмыкия,Лаганское городское муниципальное образование,Лагань,Революционная улица,1,NaN,False
8679,8809.0,496.5,"ZHIGULSKOGO 1""Z"" LAGAN","Россия, Республика Калмыкия, Лагань, улица Жиг...",47.361688,45.389283,Россия,Республика Калмыкия,Лаганское городское муниципальное образование,Лагань,улица Жигульского,1,0.036916,True
8680,8810.0,496.5,GORODOVIKOVA 3A ELISTA,"Россия, Республика Калмыкия, Элиста, улица Б. ...",44.266498,46.306946,Россия,Республика Калмыкия,городской округ Элиста,Элиста,улица Б. Городовикова,3А,-0.135684,True


In [9]:
# Создаем копию датасета
df_points = final_df.copy()

# Создаём столбец с текстом для всплывающей подсказки
df_points["__popup__"] = df_points.apply(lambda row: f"ID: {row['id']}. {row['geo_address']}", axis=1)

# Формируем список точек (lat, lon, popup)
points = list(zip(df_points["geo_lat"], df_points["geo_lon"], df_points["__popup__"]))

# Создаем карту и отображаем ее
atm_map_clean = yandex_folium_map(points)
atm_map_clean.save("data/yandex_map_clean_dataset.html")
webbrowser.open(f"file://{os.path.abspath('data/yandex_map_clean_dataset.html')}")

True

## Дополнительный блок для работы с очищенными геоданными

Ниже приведён код, который:
1. Читает итоговый CSV-файл с очищенными адресами
2. Отображает его содержимое
3. Позволяет удалить строку по индексу, если были выявлены ошибочные данные
4. Позволяет исправить поле `geo_address` вручную с верным адресом и затем повторно выполнить геокодирование выбранной строки с обновлением координат и компонентов адреса
5. В конце можно просмотреть изменённую строку и сохранить файл с изменениями

Даннвй блок можно использовать как «страховочную» ячейку при работе с финальным набором данных


In [ ]:
# Чтение и просмотр файла

# Путь к очищенному датасету
INPUT_PATH = "data/geo_data_cleaned.csv"

# Чтение файла
cleaned_df = pd.read_csv(INPUT_PATH, encoding="utf-8")

# Отображение датасета
display(cleaned_df)

In [ ]:
# Удаление строки по индексу

# Индекс строки для удаления
INDEX_TO_DELETE = None

# Удаление строки
if INDEX_TO_DELETE is not None:
    cleaned_df.drop(index=INDEX_TO_DELETE, inplace=True, errors="ignore")
    cleaned_df.reset_index(drop=True, inplace=True) # Обновление индексов

# Отображение датасета
display(cleaned_df)

In [ ]:
# Исправление строки по индексу

# Индекс строки для исправления
INDEX_TO_CHANGE = None

# Новое значение для поля geo_address, по которому затем произойдет повторное геокодирование
NEW_VALUE = "улица Металлургов, 8, Заринск, Алтайский край"

if INDEX_TO_CHANGE is not None:
    # Обновление поля geo_address
    cleaned_df.at[INDEX_TO_CHANGE, "geo_address"] = NEW_VALUE

    # API ключ для Yandex HTTP Геокодер
    # Можно получить тут - https://developer.tech.yandex.ru/services/3
    API_KEY = None

    # Проверяем, что ключ задан
    if API_KEY is not None:
        # Результат после геокодирования и замена значений в датасете
        result = geocode(cleaned_df.at[INDEX_TO_CHANGE, "geo_address"], API_KEY)

        if not result:
            print(f"Геокодер не вернул результат для индекса {INDEX_TO_CHANGE}!")
        else:
            cleaned_df.at[INDEX_TO_CHANGE, "geo_address"] = result["text"]
            cleaned_df.at[INDEX_TO_CHANGE, "geo_lon"] = result["lon"]
            cleaned_df.at[INDEX_TO_CHANGE, "geo_lat"] = result["lat"]
            for kind_dict in result["adress_components"]:
                component = kind_dict["kind"]
                value = kind_dict["name"]
                cleaned_df.at[INDEX_TO_CHANGE, f"{component}"] = value
    
    # Отображение измененной строки
    display(cleaned_df.loc[INDEX_TO_CHANGE])

In [ ]:
# Сохранение исправленного датасета

# Путь для сохранения датасета
OUTPUT_PATH = "data/geo_data_cleaned.csv"

cleaned_df = cleaned_df.sort_values(by="id").reset_index(drop=True)
cleaned_df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8")